In [1]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

from DataCombine_neg import GetData
from ourMethod_gpu import Model
from newNCTF_NN_3 import NCTF_ConvKAN_16, NCTF_ConvKAN_16_noeca1, NCTF_ConvKAN_16_noeca2, NCTF_ConvKAN_16_noeca12
from newNCTF_NN_3 import NCTF_ConvMLP_16
from newNCTF_NN_3 import NCTF_ConvKAN_13, NCTF_ConvKAN_1, ConvKAN
from newNCTF_NN_3 import NCTF_ConvKAN_17, NCTF_ConvKAN_11, ConvKAN_1
from newNCTF_NN_3 import Costco
from newNCTF_NN_3 import CTF_DDI
from newNCTF_NN_3 import DeepSynergy_new, DTF_new
from newNCTF_NN_3 import NCTF_ConvMLP_16_noeca12
from newNCTF_NN_3 import NCTF_ConvMLP_16_noeca2
from newNCTF_NN_3 import NCTF_ConvKAN_CBAM
from newNCTF_NN_3 import NCTF_ConvKAN_18,NCTF_ConvMLP_18,NCTF_KAN_18,NCTF_MLP_18
from newNCTF_NN_3 import NCTF_ConvKAN_19
from newNCTF_NN_3 import NCTF_ConvKAN_20
from compareNumpyMethod import Model as numpyModel
import pandas as pd
import os
from torch.backends import cudnn
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
import math
import time
import tensorly as tl
from tqdm import tqdm
# from utils import draw

tl.set_backend('pytorch')

def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

# def he_init_1(m):
#     if isinstance(m, (torch.nn.Linear, torch.nn.Conv1d, torch.nn.Conv2d)):
#         torch.nn.init.kaiming_normal_(m.weight, mode = 'fan_in')

def he_init_1(m):
    if isinstance(m, (torch.nn.Linear, torch.nn.Conv1d, torch.nn.Conv2d)):
        #torch.nn.init.kaiming_normal_(m.weight, mode = 'fan_in')
        torch.nn.init.kaiming_normal_(m.weight, a=0, mode = 'fan_in', nonlinearity='relu')
        #torch.nn.init.kaiming_normal_(m.weight, a=0.01, mode = 'fan_in', nonlinearity='leaky_relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

def train_test(model, criterion, lr, epochs, train_loader, val_loader, idxs_test, labels_test,k,k_folds,device):
    print(model)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
    # 训练模型
    # loss_train_list = []
    # loss_test_list = []
    for epoch in tqdm(range(epochs)):
    #for epoch in range(epochs):
        ##训练
        model.train()
        train_loss, valid_loss = 0, 0
        # loss_train_list_batch = []
        for inputs in train_loader:
            optimizer.zero_grad()
            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
            outputs = model(inputs_gpu)
            #print(inputs[-1].unsqueeze(1))
            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(inputs)
        train_loss /= len(train_loader.dataset)
        # loss_train_list.append(train_loss)

        # 验证模型
        model.eval()
        val_loss = 0.0
        for inputs in val_loader:
            with torch.no_grad():
                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                outputs = model(inputs_gpu)
                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                val_loss += loss.item() * len(inputs)
        val_loss /= len(val_loader.dataset)
        # loss_test_list.append(val_loss)

        # if epoch % 5 == 0:
        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")

        if min_val <= val_loss and epoch - min_epoch >= 10:
            break

        if min_val > val_loss:
            min_val = val_loss
            min_epoch = epoch
            # torch.save(Neural_Model, './best_model.pt')
            testModel = model

    # draw(loss_train_list, loss_test_list, str(k+1) + '-loss.png')
    # print('Finished Training.\nK-fold, Epoch, min val_loss ({},{},{})'.format(k, min_epoch, min_val))
    # 测试模型
    testModel.eval()
    with torch.no_grad():
        inputs_gpu = idxs_test.T.to(device)
        outputs = testModel(inputs_gpu)
        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
    return outputs,testModel

class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', msi=10, times=10,folds=5,a=0.5,negs=1,
                 lr=0.001, epoch=150, batch_size=2048, nc=57,
                 kernel_size=[(1, 3), (57, 1)], dims=[1],
                 **kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.numpyModel = numpyModel(model_name)
        self.msi = msi
        self.times = times
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.channel = nc
        self.shape = drug_drug_data.X.shape
        self.kernel_size = kernel_size
        self.dims = dims
        self.folds = folds
        self.a = a
        self.negs = negs
        self.parameters = kwargs

    def CV_triplet(self):
        k_folds = self.folds
        fix_seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        kname = ['kernel1', 'kernel2','kernel3']
        dname = ['dims1', 'dims2', 'dima3']
        kernel_sizeList = [[(1, len(self.shape)), (r, 1)], [(r, 1), (1, len(self.shape))]]  # our
        #kernel_sizeList = [[(1, len(self.shape)), (r, 1)], [(r, 1), (1, len(self.shape))], [(1, 1),(1, len(self.shape)), (r, 1)]]
        dimsList = [[1], [self.channel, 1], [self.channel, self.channel, 1]]  # pre层
        s1=kname[kernel_sizeList.index(self.kernel_size)]
        s2=dname[dimsList.index(self.dims)]
        df = pd.DataFrame(columns=['j', 'methods', 'times', 'folds', 'kernel', 'dims', 'aupr', 'auc', 'f1_score', 'accuracy',
                     'recall', 'specificity',
                     'precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))

            for k in range(k_folds):
                ### Train data
                # posIndex_train = torch.tensor(torch.nonzero(train_X == 1), dtype=torch.int)
                posIndex_train = torch.tensor(index_matrix[:, np.where(poscv != k)[0]]).T
                negIndex_train = torch.tensor(neg_matrix[:, np.where(negcv != k)[0]]).T
                idxs_train = torch.cat((posIndex_train, negIndex_train), dim=0)
                # print(idxs_train)
                # print(idxs_train.shape)
                poslabel_train = torch.ones(posIndex_train.shape[0])
                neglabel_train = torch.zeros(negIndex_train.shape[0])
                labels_train = torch.cat((poslabel_train, neglabel_train), dim=0)
                # print(labels_train.shape)

                ### 划分验证集
                idxs = idxs_train.numpy().astype(int)
                vals = labels_train.numpy().astype(float)
                # print(idxs,vals)
                # print(idxs.shape, vals.shape)
                idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1)
                # idxs_train, idxs_val, labels_train, labels_val = train_test_split(idxs_train, labels_train, test_size=0.1)

                ### Test data
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = torch.cat((posIndex_test, negIndex_test), dim=0)
                # print(idxs_test)
                # print(idxs_test.shape)
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                # print(labels_test.shape)

                # 模型超参数
                shape = self.drug_drug_data.X.shape
                rank = self.parameters['r']
                nc = self.channel
                kernel_size = self.kernel_size  # 0-our 1-costco原始设置
                # kernel_size2= [(rank,1),(1,len(shape))] #costco原始设置
                dims = self.dims
                lr = self.lr
                epochs = self.epoch
                batch_size = self.batch_size
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                # 创建数据加载器
                idxs_train1 = torch.LongTensor(idxs_train1)
                idxs_val = torch.LongTensor(idxs_val)
                # idxs_test = torch.LongTensor(idxs_test)
                labels_train1 = torch.FloatTensor(labels_train1)
                labels_val = torch.FloatTensor(labels_val)
                # labels_test = torch.FloatTensor(labels_test)

                train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                del train_dataset, val_dataset

                msi = [1, 2,
                       3,4,5,
                       6,
                       7,8,9,10,
                       11,12,
                          13,14,15,16,17,18]
                mnameList = ['NCTF_ConvKAN_16','NCTF_ConvMLP_16',
                             'NCTF_ConvKAN_16_noeca1','NCTF_ConvKAN_16_noeca2','NCTF_ConvKAN_16_noeca12',
                             'NCTF_ConvKAN_17',
                             'NCTF_ConvMLP_16_noeca12','NCTF_ConvKAN_13','NCTF_ConvKAN_1','ConvKAN',
                             'NCTF_ConvMLP_16_noeca2','NCTF_ConvKAN_CBAM',
                            'NCTF_ConvKAN_18','NCTF_ConvKAN_19','NCTF_ConvKAN_20','NCTF_ConvMLP_18','NCTF_KAN_18','NCTF_MLP_18']
                mname = mnameList[msi.index(self.msi)]
                print(mname)

                ##### 获取NCTF学习所得的因子矩阵 M, C, D #####
                ### end to end 学习M C D
                # train_tensor = np.array(self.drug_drug_data.X, copy=True)
                # trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                # train_tensor[trainpos_index] = 0
                # train_X = torch.tensor(train_tensor, dtype=torch.float32)
                # S1 = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                # S2 = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                # _, M, C, D = self.model()(train_X, S1, S2,
                #                           r=self.parameters['r'],
                #                           mu=self.parameters['mu'], eta=self.parameters['eta'],
                #                           alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                #                           lam=self.parameters['lam'],
                #                           tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                #                           )
                # print('NCTF')

                ### 直接导入提前学习好的因子矩阵 
                fname='NCTF_embeds/'+str(self.negs) +'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                print(fname)
                with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
                    M, C, D = pickle.load(f)
                M = M.to(device)
                C = C.to(device)
                D = D.to(device)
                
                print(M.shape, C.shape, D.shape)
                
                ### 构建深度非线性模型 ### 定义损失函数 和 优化器
                if self.msi == 1:
                    Neural_Model = NCTF_ConvKAN_16(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 2:
                    Neural_Model = NCTF_ConvMLP_16(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.ReLU(), dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 3:
                    Neural_Model = NCTF_ConvKAN_16_noeca1(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)
                elif self.msi == 4:
                    Neural_Model = NCTF_ConvKAN_16_noeca2(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)
                elif self.msi == 5:
                    Neural_Model = NCTF_ConvKAN_16_noeca12(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)
                    
                elif self.msi == 6:
                    Neural_Model = NCTF_ConvKAN_17(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 7:
                    Neural_Model = NCTF_ConvMLP_16_noeca12(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.ReLU(), dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 8:
                    Neural_Model = NCTF_ConvKAN_13(shape, rank, nc, M, C, D, device, kernel_size,
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 9:
                    Neural_Model = NCTF_ConvKAN_1(shape, rank, nc, M, C, D, device, kernel_size,
                                                  dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 10:
                    Neural_Model = ConvKAN(shape, rank, nc, device,
                                           kernel_size=kernel_size, dims=dims,
                                           act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 11:
                    Neural_Model = NCTF_ConvMLP_16_noeca2(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.ReLU(), dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 12:
                    Neural_Model = NCTF_ConvKAN_CBAM(shape, rank, M, C, D, device, kernel_size,nc=[nc,nc],
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 13:
                    Neural_Model = NCTF_ConvKAN_18(shape, rank, nc, M, C, D, device, kernel_size,
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0,alpha = self.a).to(device)

                elif self.msi == 14:
                    sim=[S1.to(device),S1.to(device),S2.to(device)]
                    Neural_Model = NCTF_ConvKAN_19(shape, rank, M, C, D, device, sim, kernel_size,nc=[nc,nc],
                                                  dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 15:
                    sim=[S1.to(device),S1.to(device),S2.to(device)]
                    Neural_Model = NCTF_ConvKAN_20(shape, rank, M, C, D, device, sim, kernel_size,nc=[nc,nc],
                                                  dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0).to(device)

                elif self.msi == 16:
                    Neural_Model = NCTF_ConvMLP_18(shape, rank, nc, M, C, D, device, kernel_size,
                                                   dims=dims, act_func=nn.ReLU(), dropout=0.0, input_dropout=0.0,alpha = self.a).to(device)

                elif self.msi == 17:
                    Neural_Model = NCTF_KAN_18(shape, rank, nc, M, C, D, device, kernel_size,
                                                   dims=dims, act_func=nn.SiLU, dropout=0.0, input_dropout=0.0,alpha = self.a).to(device)

                elif self.msi == 18:
                    Neural_Model = NCTF_MLP_18(shape, rank, nc, M, C, D, device, kernel_size,
                                                   dims=dims, act_func=nn.ReLU(), dropout=0.0, input_dropout=0.0,alpha = self.a).to(device)

                # print(Neural_Model)
                #criterion = nn.MSELoss()
                Neural_Model.apply(he_init_1)
                criterion = nn.BCEWithLogitsLoss()
                # optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)
                outputs,testModel = train_test(Neural_Model, criterion, lr, epochs, train_loader, val_loader, idxs_test, labels_test, k, k_folds, device)

                ### 存储每折每次的预测和真实值
                # fname='newscore/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores_mse.pkl'
                # print(fname)
                # with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                #     pickle.dump([labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([testModel,idxs_test,labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                # print(idxs_test.T)
                # print(idxs_test.T[0],len(idxs_test.T[0]))
                results = pd.DataFrame({
                    'time': [i] * len(idxs_test.T[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test.T[0]),  # 假设这是第 1 次
                    'm1': idxs_test.T[0],
                    'm2': idxs_test.T[1],
                    'd': idxs_test.T[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': outputs.T[0].cpu().numpy()  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)
                
                ### 计算评价指标
                metrics = self.get_metrics_1(labels_test.cpu().numpy(), outputs.T[0].cpu().numpy())
                # print(metrics)
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                # print(metrics)
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, mname, i, k, kernel_size, dims, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j = j + 1

            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t', i + 1, ':\t', result)
            avgmetrics_tensor_10 = avgmetrics_tensor_10 + result

        #print(self.a,str(self.a))
        sname = s1 + '_' + s2 + '_' + str(self.a)
        # fname = os.path.join('newablation/new3', mname + '_' + sname + '_hmddv32_5times5CV_1neg_results_bceheinit.csv')
        # print(fname)
        # df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        fname = os.path.join('compareTF', mname + '_' + sname + '_hmddv3.2_'+str(self.negs)+'neg_results_bceheinit_fixed_new.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        # print(j)
        # print(df)
        # print(metrics_tensor_all)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t', results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t', results_2)
        return results_1


    def get_metrics_1(self, real_score, predict_score):
        real_score = np.mat(real_score)
        predict_score = np.mat(predict_score)
        # print(real_score)
        # print(real_score.shape)
        # print(predict_score)
        # print(predict_score.shape)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        # sorted_predict_score = np.array(sorted(list(set(predict_score))))
        # print(sorted_predict_score)
        # print(sorted_predict_score.shape)
        # print(np.array(real_score).flatten())
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]

        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1

        # print(real_score.T)
        # print(real_score.T.shape)
        # print(np.mat(real_score).T)
        # print(np.mat(real_score).T.shape)
        # print(predict_score_matrix.shape)
        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN

        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T

        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])

        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])

        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)

        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]

        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision
        
    def get_metrics_2(self, real_score, predict_score):
        np.random.seed(2024)
        # trues, preds =  np.array(real_score).flatten(),np.array(predict_score).flatten()
        print(predict_score)
        trues, preds = real_score, expit(predict_score)
        print(preds)
        # print(preds, trues)
        fpr1, tpr1, thresholds1 = roc_curve(trues, preds, pos_label=1)
        auc_value = auc(fpr1, tpr1)
        # print(thresholds1)
        # auc_value = roc_auc_score(trues, preds)
        precision, recall, thresholds2 = precision_recall_curve(trues, preds, pos_label=1)
        precision = precision + np.finfo(float).tiny  # 添加极小值防止出现0
        recall = recall + np.finfo(float).tiny  # 添加极小值防止出现0
        # print(thresholds2)

        f1_scores = 2 * (precision * recall) / (precision + recall)
        best_f1_index = f1_scores.argmax()
        best_f1 = f1_scores[best_f1_index]
        best_threshold = thresholds2[best_f1_index]
        print('Best F1 Score:', best_f1)
        print('Best Threshold:', best_threshold)
        best_recall = recall[best_f1_index]
        best_precision = precision[best_f1_index]
        print(best_recall, best_precision)
        f1_scores = 2 * (best_precision * best_recall) / (best_precision + best_recall)
        print(f1_scores)
        print(best_recall, best_precision, f1_scores)

        # best_threshold = np.median(thresholds1) # 中位数

        aupr = average_precision_score(trues, preds, pos_label=1)
        preds1 = preds
        preds1[preds > best_threshold] = 1
        preds1[preds <= best_threshold] = 0

        labels = [1]
        TP, FP, FN, TN = 0, 0, 0, 0
        for label in labels:
            preds_tmp = np.array([1 if pred == label else 0 for pred in preds1])
            trues_tmp = np.array([1 if true == label else 0 for true in trues])
            # print(preds_tmp, trues_tmp)
            # print()
            # TP预测为1真实为1
            # TN预测为0真实为0
            # FN预测为0真实为1
            # FP预测为1真实为0
            TP += ((preds_tmp == 1) & (trues_tmp == 1)).sum()
            TN += ((preds_tmp == 0) & (trues_tmp == 0)).sum()
            FN += ((preds_tmp == 0) & (trues_tmp == 1)).sum()
            FP += ((preds_tmp == 1) & (trues_tmp == 0)).sum()
        print(TP, FP, FN, TN)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1_score = 2 * precision * recall / (precision + recall)
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        specificity = TN / (TN + FP)
        npre = TN / (TN + FN)
        fpr = FP / (TN + FP)
        fnr = FN / (TP + FN)
        print(recall, precision, f1_score)

        return aupr, auc_value, f1_score, accuracy, recall, specificity, precision

    def get_metrics_3(self, real_score, predict_score):
        # fpr1, tpr1, thresholds = roc_curve(trues, preds, pos_label=1)
        # auc_value = auc(fpr1, tpr1)
        print(predict_score)
        trues, preds = real_score, expit(predict_score)
        print(preds)
        auc_value = roc_auc_score(trues, preds)
        # precision1, recall1, _ = precision_recall_curve(trues, preds, pos_label=1)
        aupr = average_precision_score(trues, preds, pos_label=1)
        preds1 = preds
        # preds1[preds1 > 0.5] = 1
        # preds1[preds1 <= 0.5] = 0
        preds1[preds1 > np.median(preds)] = 1
        preds1[preds1 <= np.median(preds)] = 0

        labels = [1]
        TP, FP, FN, TN = 0, 0, 0, 0
        for label in labels:
            preds_tmp = np.array([1 if pred == label else 0 for pred in preds1])
            trues_tmp = np.array([1 if true == label else 0 for true in trues])
            print(preds_tmp, trues_tmp)
            # print()
            # TP预测为1真实为1
            # TN预测为0真实为0
            # FN预测为0真实为1
            # FP预测为1真实为0
            TP += ((preds_tmp == 1) & (trues_tmp == 1)).sum()
            TN += ((preds_tmp == 0) & (trues_tmp == 0)).sum()
            FN += ((preds_tmp == 0) & (trues_tmp == 1)).sum()
            FP += ((preds_tmp == 1) & (trues_tmp == 0)).sum()
        print(TP, FP, FN, TN)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = 2 * precision * recall / (precision + recall)
        accuracy = (TP + TN) / (TP + TN + FP + FN)
        specificity = TN / (TN + FP)
        npre = TN / (TN + FN)
        fpr = FP / (TN + FP)
        fnr = FN / (TP + FN)

        return aupr, auc_value, f1, accuracy, recall, specificity, precision


if __name__ == '__main__':
    fix_seed(2024)
    ### 导入数据
    since = time.time()
    ###循环次数
    times = 5
    ### 导入数据
    #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    # signal = 11
    # miRNA_num = 351
    # disease_num = 325
    # neg = 1
    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
    # #folder = '/mnt/sda/liupei/NCTF/newCode/data/ddi5cv'
    # #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    # drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal,neg=neg)
    ## 设置参数
    lr = 0.0001  ## 设置均不同
    batch_size = 256 #1024
    epoch = 500
    shape = 3
    r = 57
    nc = int(2*r)
    kernel_sizeList = [[(1, shape), (r, 1)], [(r, 1), (1, shape)]]  # our
    dimsList = [[1], [nc, 1], [nc, nc, 1]]  # pre层
    ### 搜索最优配置
    # msiList = [1, 2, 310, 311, 312, 313, 314,315,316,320, 321, 322, 323,324,325,326,327]
    # mnameList = ['ConvMLP', 'ConvKAN',
    #              'NCTF_ConvMLP', 'NCTF_ConvMLP_1', 'NCTF_ConvMLP_12', 'NCTF_ConvMLP_13','NCTF_ConvMLP_14', 'NCTF_ConvMLP_15','NCTF_ConvMLP_16',
    #              'NCTF_ConvKAN', 'NCTF_ConvKAN_1', 'NCTF_ConvKAN_12', 'NCTF_ConvKAN_13', 'NCTF_ConvKAN_14', 'NCTF_ConvKAN_15',
    #              'NCTF_ConvKAN_16', 'NCTF_ConvKAN_17']
    # msiList = [1, 2,
    #            3,4,5,
    #            6]
    # mnameList = ['NCTF_ConvKAN_16','NCTF_ConvMLP_16',
    #              'NCTF_ConvKAN_16_noeca1','NCTF_ConvKAN_16_noeca2','NCTF_ConvKAN_16_noeca12',
    #              'NCTF_ConvKAN_17']
    msiList = [1, 2,
               3,4,5,
               6,
               7,8,9,10,
               11,
              12,13,14,15,16,17,18]
    mnameList = ['NCTF_ConvKAN_16','NCTF_ConvMLP_16',
                 'NCTF_ConvKAN_16_noeca1','NCTF_ConvKAN_16_noeca2','NCTF_ConvKAN_16_noeca12',
                 'NCTF_ConvKAN_17',
                 'NCTF_ConvMLP_16_noeca12','NCTF_ConvKAN_13','NCTF_ConvKAN_1','ConvKAN',
                 'NCTF_ConvMLP_16_noeca2',
                'NCTF_ConvKAN_CBAM','NCTF_ConvKAN_18','NCTF_ConvKAN_19','NCTF_ConvKAN_20','NCTF_ConvMLP_18','NCTF_KAN_18','NCTF_MLP_18']
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    i = 0
    kernel_size = kernel_sizeList[0]
    #dims = dimsList[0]
    folds = 5
    mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001
    #mu,eta,alpha,beta,lam=0.25,2,0.125,0.125,0.001
    a=0.5
    signal = 11
    for msi in [13]:
        mname = mnameList[msiList.index(msi)]
        print(mname)
        # if msi in [2,7,11,16]:
        #     dims=dimsList[1]
        # else:
        #     dims=dimsList[0]
        dims = dimsList[0]# one-layer
        #for neg in [1,2,4,6,8,10]:
        for neg in [1]:
            folder = '/mnt/sda/liupei/NCTF_new/data/hmddv32_neg/'+str(neg)+'n'
            drug_drug_data = GetData(miRNA_num=351, disease_num=325,filefolder=folder,signal=signal,neg=neg)
            since1 = time.time()
            experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32', msi=msi, times=times, folds=folds,a=a,negs=neg,
                                     lr=lr, epoch=epoch, batch_size=batch_size, nc=nc,
                                     kernel_size=kernel_size, dims=dims,
                                     r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol = 1e-4, max_iter = 100)
    
            aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
            df.loc[i] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity,precision]
            print(f"neg={neg}")
            print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
            i = i + 1
            time_elapsed1 = time.time() - since1
            print(time_elapsed1 // 60, time_elapsed1 % 60)

    df.to_csv('NCTFConvKAN18_negResults_fixed_new.csv',index=False)
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)

/home/liupei/miniconda3/envs/pyg/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


NCTF_ConvKAN_18
(351, 351)
(325, 325)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
NCTF_ConvKAN_18
NCTF_embeds/1n/factors_0_times_0_fold.pkl
torch.Size([351, 57]) torch.Size([351, 57]) torch.Size([325, 57])
NCTF_ConvKAN_18(
  (embeds1): Embedding(351, 57)
  (embeds2): Embedding(325, 57)
  (conv1): Conv2d(1, 114, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(57, 1), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (act): ReLU()
  (output): KANLayers(
    (layers): ModuleList(
      (0): FastKANLayer(
        (layernorm): LayerNorm((114,), eps=1e-05, elementwise_affine=True)
        (rbf): RadialBasisFunction()
        (spline_linear): SplineLinear(in_features=912, out_features=1, bias=False)
        (base_activation): SiLU()
        (base_linear): Linear(in_features=114, out_features=1, bias=True)
      )
    )
  )
)


 10%|███████████▉                                                                                                           | 50/500 [00:27<04:07,  1.82it/s]


Fold 1/5, Test Loss: 0.028767
pred_score_pkl/NCTF_ConvKAN_18_0_times_0_foldscores.pkl
NCTF_ConvKAN_18
NCTF_embeds/1n/factors_0_times_1_fold.pkl
torch.Size([351, 57]) torch.Size([351, 57]) torch.Size([325, 57])
NCTF_ConvKAN_18(
  (embeds1): Embedding(351, 57)
  (embeds2): Embedding(325, 57)
  (conv1): Conv2d(1, 114, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(57, 1), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (act): ReLU()
  (output): KANLayers(
    (layers): ModuleList(
      (0): FastKANLayer(
        (layernorm): LayerNorm((114,), eps=1e-05, elementwise_affine=True)
        (rbf): RadialBasisFunction()
        (spline_linear): SplineLinear(in_features=912, out_features=1, bias=False)
        (base_activation): SiLU()
        (base_linear): Linear(in_features=114, out_features=1, bias=True)
      )
    )
  )
)


 17%|████████████████████▋                                                                                                  | 87/500 [00:45<03:37,  1.90it/s]


...

Fold 3/5, Test Loss: 0.026525
pred_score_pkl/NCTF_ConvKAN_18_4_times_2_foldscores.pkl
NCTF_ConvKAN_18
NCTF_embeds/1n/factors_4_times_3_fold.pkl
torch.Size([351, 57]) torch.Size([351, 57]) torch.Size([325, 57])
NCTF_ConvKAN_18(
  (embeds1): Embedding(351, 57)
  (embeds2): Embedding(325, 57)
  (conv1): Conv2d(1, 114, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(57, 1), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (act): ReLU()
  (output): KANLayers(
    (layers): ModuleList(
      (0): FastKANLayer(
        (layernorm): LayerNorm((114,), eps=1e-05, elementwise_affine=True)
        (rbf): RadialBasisFunction()
        (spline_linear): SplineLinear(in_features=912, out_features=1, bias=False)
        (base_activation): SiLU()
        (base_linear): Linear(in_features=114, out_features=1, bias=True)
      )
    )
  )
)


 15%|█████████████████▊                                                                                                     | 75/500 [00:38<03:40,  1.93it/s]


Fold 4/5, Test Loss: 0.031369
pred_score_pkl/NCTF_ConvKAN_18_4_times_3_foldscores.pkl
NCTF_ConvKAN_18
NCTF_embeds/1n/factors_4_times_4_fold.pkl
torch.Size([351, 57]) torch.Size([351, 57]) torch.Size([325, 57])
NCTF_ConvKAN_18(
  (embeds1): Embedding(351, 57)
  (embeds2): Embedding(325, 57)
  (conv1): Conv2d(1, 114, kernel_size=(1, 3), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(57, 1), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (act): ReLU()
  (output): KANLayers(
    (layers): ModuleList(
      (0): FastKANLayer(
        (layernorm): LayerNorm((114,), eps=1e-05, elementwise_affine=True)
        (rbf): RadialBasisFunction()
        (spline_linear): SplineLinear(in_features=912, out_features=1, bias=False)
        (base_activation): SiLU()
        (base_linear): Linear(in_features=114, out_features=1, bias=True)
      )
    )
  )
)


 13%|███████████████▏                                                                                                       | 64/500 [00:33<03:48,  1.91it/s]

Fold 5/5, Test Loss: 0.025770
pred_score_pkl/NCTF_ConvKAN_18_4_times_4_foldscores.pkl
Times:	 5 :	 [[0.9985 0.9988 0.9922 0.9922 0.9933 0.9911 0.9911]]
final:	 [[0.9988 0.9991 0.9926 0.9926 0.9946 0.9905 0.9906]]
neg=1
auc=0.9991	aupr=0.9988	f1=0.9926	acc=0.9926	recall=0.9946	spe=0.9905	pre=0.9906

16.0 16.173750638961792
16.0 16.380422115325928


In [2]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

from DataCombine_neg import GetData
from ourMethod_gpu import Model
from newNCTF_NN_2 import NCTF_ConvKAN_16, NCTF_ConvKAN_16_noeca1, NCTF_ConvKAN_16_noeca2, NCTF_ConvKAN_16_noeca12
from newNCTF_NN_2 import NCTF_ConvMLP_16
from newNCTF_NN_2 import NCTF_ConvKAN_13, NCTF_ConvKAN_1, ConvKAN
from newNCTF_NN_2 import NCTF_ConvKAN_17, NCTF_ConvKAN_11, ConvKAN_1
from newNCTF_NN_2 import Costco
from newNCTF_NN_2 import CTF_DDI
from newNCTF_NN_2 import DeepSynergy_new, DTF_new
from compareNumpyMethod import Model as numpyModel
#from DataCombine_drug import GetData
import pandas as pd
import os
from torch.backends import cudnn
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
import math
import time
import tensorly as tl
from tqdm import tqdm
import gzip


# from utils import draw

# tl.set_backend('pytorch')

def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

def normalize_Deepsynergy(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt)      

def normalize_DTF(X, means1=None, std1=None, means2=None, std2=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    X = np.ascontiguousarray(X)
    if norm is None:
        return (X, means1, std1, feat_filt)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1
    if norm == 'norm':
        return(X, means1, std1)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        return(X, means1, std1, means2, std2)
        
class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', msi=10, times=10, negs=1,
                 lr=0.001, epoch=150, batch_size=2048, nc=57,
                 **kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.numpyModel = numpyModel(model_name)
        self.msi = msi
        self.times = times
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.channel = nc
        self.negs = negs
        self.parameters = kwargs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = 5
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'methods', 'times', 'folds', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity','precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))

            for k in range(k_folds):
                ### Train data
                # posIndex_train = torch.tensor(torch.nonzero(train_X == 1), dtype=torch.int)
                posIndex_train = torch.tensor(index_matrix[:, np.where(poscv != k)[0]]).T
                negIndex_train = torch.tensor(neg_matrix[:, np.where(negcv != k)[0]]).T
                idxs_train = torch.cat((posIndex_train, negIndex_train), dim=0)
                # print(idxs_train)
                # print(idxs_train.shape)
                poslabel_train = torch.ones(posIndex_train.shape[0])
                neglabel_train = torch.zeros(negIndex_train.shape[0])
                labels_train = torch.cat((poslabel_train, neglabel_train), dim=0)
                # print(labels_train.shape)

                ### 划分验证集
                idxs = idxs_train.numpy().astype(int)
                vals = labels_train.numpy().astype(float)
                # print(idxs,vals)
                # print(idxs.shape, vals.shape)
                idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                # idxs_train, idxs_val, labels_train, labels_val = train_test_split(idxs_train, labels_train, test_size=0.1)

                ### Test data
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = torch.cat((posIndex_test, negIndex_test), dim=0)
                # print(idxs_test)
                # print(idxs_test.shape)
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                # print(labels_test.shape)

                # 模型超参数
                shape = self.drug_drug_data.X.shape
                rank = self.parameters['r']
                nc = self.channel
                lr = self.lr
                epochs = self.epoch
                batch_size = self.batch_size
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                methodList = [20, 70, 80, 90]
                mnameList = ['Costco', 'DeepSynergy', 'DTF', 'CTF_DDI']

                ### 构建深度非线性模型 ### 定义损失函数 和 优化器
                if self.msi == 20:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)

                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    Neural_Model = Costco(shape, rank, nc, device).to(device)  # Costco
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                elif self.msi == 70:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    ### 获得三个维度的相似度特征
                    M = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    C = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    D = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
                    # M, C, D = getSimFeature(folder=folder, signal=11)
                    # M = torch.FloatTensor(M)
                    # C = torch.FloatTensor(C)
                    # D = torch.FloatTensor(D)
                    # inputSize= (M.shape[1]+C.shape[1]+D.shape[1])
                    # print(inputSize)
                    
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)
                    norm = 'tanh'    
                    ## training 
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_val, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_tr, mean, std, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, feat_filt = normalize_Deepsynergy(X_val, mean, std, feat_filt=feat_filt, norm=norm)

                    #print(X_tr.shape,X_val.shape)
                    ## testing    
                    if norm == "tanh_norm":
                        X_train, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_test, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_train, mean, std, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, feat_filt = normalize_Deepsynergy(X_test, mean, std, feat_filt=feat_filt, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                 
                    inputSize = X_tr.shape[1]
                    Neural_Model = DeepSynergy_new(shape, rank, inputSize, 
                                               X_tr,X_val,X_train,X_test,
                                               act_func=nn.ReLU(),dropout=0.5, input_dropout=0.2,
                                               dims=[8182, 4096, 1]).to(device)
                    # [8182, 4096, 1];[4096, 2048, 1]
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    Neural_Model.apply(he_init)
                    # optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)
                    optimizer = optim.SGD(Neural_Model.parameters(), lr=lr, momentum=0.5)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                        # label_pre=outputs
                        # label_pre[ label_pre < threshold ] = 0
                        # label_pre[ label_pre >= threshold ] = 1
                        # outputs[ outputs < 30 ] = 0
                        # outputs[ outputs >= 30 ] = 1
                        # labels_test [labels_test<30] = 0
                        # labels_test [labels_test >=30] =1
                        #print(outputs,labels_test)

                elif self.msi == 80:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    fname = os.path.join('./cpwopt_57R/output/', 'embM_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    M = emb.values
                    #M = torch.FloatTensor(M).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embC_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    C = emb.values
                    #C = torch.FloatTensor(C).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embD_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    D = emb.values
                    #D = torch.FloatTensor(D).to(device)
                    print(M.shape,C.shape,D.shape)

                    M = torch.FloatTensor(M)
                    C = torch.FloatTensor(C)
                    D = torch.FloatTensor(D)
                    ### 得到数据集的特征
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    ### 数据特征标准化
                    norm = 'tanh'    
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2 = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2 = normalize_DTF(X_val, mean, std, mean2, std2,  norm=norm)
                        X_test, mean, std, mean2, std2 = normalize_DTF(X_test, mean, std, mean2, std2, norm=norm)    
                    else:
                        X_tr, mean, std = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std = normalize_DTF(X_val, mean, std, norm=norm)
                        X_test, mean, std = normalize_DTF(X_test, mean, std, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    print(labels_train1,labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    ## 3000
                    #inputSize = rank * len(shape)
                    inputSize = X_tr.shape[1]
                    Neural_Model = DTF_new(shape, rank, inputSize, embeds=[M, C, D], nn_struc=[2048, 1024, 512],
                                       input_dp=0.2, first_dp=0.5, second_dp=0.5).to(device)
                    print(Neural_Model)
                    Neural_Model.apply(he_init)
                    criterion = nn.BCELoss()  # binary crossentropy loss
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)
                    
                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            #print(inputs[-1])
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    #labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                
                elif self.msi == 90:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset

                    # tl.set_backend('numpy')
                    # train_tensor = np.array(self.drug_drug_data.X, copy=True)
                    # trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                    # train_tensor[trainpos_index] = 0
                    # S1 = np.mat(self.drug_drug_data.S1)
                    # S2 = np.mat(self.drug_drug_data.S2)
                    # _, M, C, D = self.numpyModel()(train_tensor, S1, S2,
                    #                                r=self.parameters['r'],
                    #                                mu=self.parameters['mu'], eta=self.parameters['eta'],
                    #                                alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                    #                                lam=self.parameters['lam'],
                    #                                tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                    #                                )
                    # print('CTF')
                    # # print(M.shape, C.shape, D.shape)
                    # M = torch.FloatTensor(M).to(device)
                    # C = torch.FloatTensor(C).to(device)
                    # D = torch.FloatTensor(D).to(device)
                    # print(M.shape, C.shape, D.shape)
                    # print(M)

                    ### 直接导入提前学习好的因子矩阵 
                    fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    #fname='/mnt/sda/liupei/NCTF/newCode/hmddv32/compare/neg/CTF_embeds/1n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
                        M, C, D = pickle.load(f)
                    
                    M = torch.FloatTensor(M).to(device)
                    C = torch.FloatTensor(C).to(device)
                    D = torch.FloatTensor(D).to(device)
                    
                    Neural_Model = CTF_DDI(shape, rank, hids_size=[256, 256, 128], embeds=[M, C, D], device=device).to(device)
                    print(Neural_Model)
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([testModel,idxs_test,labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                # print(idxs_test.T)
                # print(idxs_test.T[0],len(idxs_test.T[0]))
                results = pd.DataFrame({
                    'time': [i] * len(idxs_test.T[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test.T[0]),  # 假设这是第 1 次
                    'm1': idxs_test.T[0],
                    'm2': idxs_test.T[1],
                    'd': idxs_test.T[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': outputs.T[0].cpu().numpy()  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)
                
                metrics = self.get_metrics_1(labels_test.cpu().numpy(), outputs.T[0].cpu().numpy())
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, mname, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j = j + 1

            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t', i + 1, ':\t', result)
            avgmetrics_tensor_10 = avgmetrics_tensor_10 + result

        fname = os.path.join('compareTF', mname + '_hmddv3.2_'+str(self.negs)+'neg_results_0.5lam_fixed.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        #print(j)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t', results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t', results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score = np.mat(real_score)
        predict_score = np.mat(predict_score)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]

        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1

        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN

        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T

        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])

        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])

        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)

        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]

        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision


def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'


if __name__ == '__main__':
    fix_seed(2024)
    ### 导入数据
    since = time.time()
    #df = pd.DataFrame(columns=['methods', 'times', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    methodList = [20, 70, 80, 90]
    mnameList = ['Costco', 'DeepSynergy','DTF', 'CTF_DDI']
    ## 设置参数
    lr = 0.0001  ## 设置均不同
    batch_size = 1024
    epoch = 500
    shape = 3
    r = 57
    nc = int(2*r)
    times = 5
    folds = 5
    #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    #signal = 11
    miRNA_num=351
    disease_num=325
    #drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
    mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    i = 0
    for msi in [90]:
        mname = mnameList[methodList.index(msi)]
        print(mname, msi)
        #for neg in [1,2,4,6,8,10]:
        for neg in [1]:
            folder = '/mnt/sda/liupei/NCTF_new/data/hmddv32_neg/'+str(neg)+'n'
            signal = 11
            print(neg)
            drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num,filefolder=folder,signal=signal,neg=neg)
            since1 = time.time()
            if msi == 20:
                ## Costco
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',msi=msi, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=500, batch_size=256, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 70:
                ### DeepSynergy DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=70, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=64, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
    
            elif msi == 80:
                ## DTF batch_size=128 r=1000
                ### DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=80, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=128, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 90:
                ## CTF_DDI batch_size=1000 epoch=300 r=51 max_iter=200
                signal = 21  # 22
                drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal,neg=neg)
                # DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='CTF',
                                         msi=90, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=300, batch_size=1000, nc=nc,
                                         r=r, mu=0.5, eta=0.2, alpha=0.5, beta=0.5,lam=0.5, tol=1e-6, max_iter=100)

            aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
            df.loc[i] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
            # experiment.CV_triplet()[0]
            print(f"i={i}\ttimes={times}\tmethods={mname}\tmsi={msi}\tneg={neg}")
            print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
            i = i + 1
            time_elapsed1 = time.time() - since1
            print(time_elapsed1 // 60, time_elapsed1 % 60)
            
    df.to_csv('CTFDDI_negResults_0.5lam_fixed.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)

CTF_DDI 90
1
(351, 351)
(325, 325)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
(351, 351)
(351, 351)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
CTF_DDI
CTF_DDI(
  (Modelist): ModuleList(
    (0): Linear(in_features=171, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:01<00:00,  4.86it/s]


Fold 1/5, Test Loss: 0.190844
tensor([[ 2.8342],
        [ 6.2300],
        [ 1.1550],
        ...,
        [-3.8365],
        [-1.5678],
        [-7.0677]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/CTF_DDI_0_times_0_foldscores.pkl
CTF_DDI
CTF_DDI(
  (Modelist): ModuleList(
    (0): Linear(in_features=171, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [01:00<00:00,  5.00it/s]


Fold 2/5, Test Loss: 0.152867
tensor([[-0.6544],
        [ 4.0768],
        [ 0.8425],
        ...,
        [ 0.0901],
        [-4.3025],
        [-7.8134]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/CTF_DDI_0_times_1_foldscores.pkl
CTF_DDI
CTF_DDI(
  (Modelist): ModuleList(
    (0): Linear(in_features=171, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


...

Fold 3/5, Test Loss: 0.132106
tensor([[-1.0292],
        [ 4.4651],
        [ 4.2320],
        ...,
        [-3.6447],
        [-8.7672],
        [-2.7568]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/CTF_DDI_4_times_2_foldscores.pkl
CTF_DDI
CTF_DDI(
  (Modelist): ModuleList(
    (0): Linear(in_features=171, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


 76%|█████████████████████████████████████████████████████████████████████████████████████████▋                            | 228/300 [00:41<00:13,  5.50it/s]


Fold 4/5, Test Loss: 0.211967
tensor([[ 3.0729],
        [ 3.8463],
        [ 0.6489],
        ...,
        [-5.6293],
        [-5.2064],
        [-4.1491]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/CTF_DDI_4_times_3_foldscores.pkl
CTF_DDI
CTF_DDI(
  (Modelist): ModuleList(
    (0): Linear(in_features=171, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=1, bias=True)
  )
)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:53<00:00,  5.61it/s]


Fold 5/5, Test Loss: 0.155012
tensor([[-1.6768],
        [ 4.9286],
        [ 3.4420],
        ...,
        [-0.4549],
        [-3.0631],
        [-2.5188]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/CTF_DDI_4_times_4_foldscores.pkl
Times:	 5 :	 [[0.9828 0.9824 0.9335 0.9333 0.9377 0.9288 0.9294]]
final:	 [[0.982  0.9813 0.9305 0.9302 0.934  0.9264 0.9271]]
i=0	times=5	methods=CTF_DDI	msi=90	neg=1
auc=0.9813	aupr=0.982	f1=0.9305	acc=0.9302	recall=0.934	spe=0.9264	pre=0.9271

22.0 29.701514720916748
22.0 29.915620803833008


In [3]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

from DataCombine_neg import GetData
from ourMethod_gpu import Model
from newNCTF_NN_2 import NCTF_ConvKAN_16, NCTF_ConvKAN_16_noeca1, NCTF_ConvKAN_16_noeca2, NCTF_ConvKAN_16_noeca12
from newNCTF_NN_2 import NCTF_ConvMLP_16
from newNCTF_NN_2 import NCTF_ConvKAN_13, NCTF_ConvKAN_1, ConvKAN
from newNCTF_NN_2 import NCTF_ConvKAN_17, NCTF_ConvKAN_11, ConvKAN_1
from newNCTF_NN_2 import Costco
from newNCTF_NN_2 import CTF_DDI
from newNCTF_NN_2 import DeepSynergy_new, DTF_new
from compareNumpyMethod import Model as numpyModel
#from DataCombine_drug import GetData
import pandas as pd
import os
from torch.backends import cudnn
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
import math
import time
import tensorly as tl
from tqdm import tqdm
import gzip


# from utils import draw

# tl.set_backend('pytorch')

def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

def normalize_Deepsynergy(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt)      

def normalize_DTF(X, means1=None, std1=None, means2=None, std2=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    X = np.ascontiguousarray(X)
    if norm is None:
        return (X, means1, std1, feat_filt)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1
    if norm == 'norm':
        return(X, means1, std1)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        return(X, means1, std1, means2, std2)
        
class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', msi=10, times=10, negs=1,
                 lr=0.001, epoch=150, batch_size=2048, nc=57,
                 **kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.numpyModel = numpyModel(model_name)
        self.msi = msi
        self.times = times
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.channel = nc
        self.negs = negs
        self.parameters = kwargs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = 5
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'methods', 'times', 'folds', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity','precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))

            for k in range(k_folds):
                ### Train data
                # posIndex_train = torch.tensor(torch.nonzero(train_X == 1), dtype=torch.int)
                posIndex_train = torch.tensor(index_matrix[:, np.where(poscv != k)[0]]).T
                negIndex_train = torch.tensor(neg_matrix[:, np.where(negcv != k)[0]]).T
                idxs_train = torch.cat((posIndex_train, negIndex_train), dim=0)
                # print(idxs_train)
                # print(idxs_train.shape)
                poslabel_train = torch.ones(posIndex_train.shape[0])
                neglabel_train = torch.zeros(negIndex_train.shape[0])
                labels_train = torch.cat((poslabel_train, neglabel_train), dim=0)
                # print(labels_train.shape)

                ### 划分验证集
                idxs = idxs_train.numpy().astype(int)
                vals = labels_train.numpy().astype(float)
                # print(idxs,vals)
                # print(idxs.shape, vals.shape)
                idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                # idxs_train, idxs_val, labels_train, labels_val = train_test_split(idxs_train, labels_train, test_size=0.1)

                ### Test data
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = torch.cat((posIndex_test, negIndex_test), dim=0)
                # print(idxs_test)
                # print(idxs_test.shape)
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                # print(labels_test.shape)

                # 模型超参数
                shape = self.drug_drug_data.X.shape
                rank = self.parameters['r']
                nc = self.channel
                lr = self.lr
                epochs = self.epoch
                batch_size = self.batch_size
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                methodList = [20, 70, 80, 90]
                mnameList = ['Costco', 'DeepSynergy', 'DTF', 'CTF_DDI']

                ### 构建深度非线性模型 ### 定义损失函数 和 优化器
                if self.msi == 20:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)

                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    Neural_Model = Costco(shape, rank, nc, device).to(device)  # Costco
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                elif self.msi == 70:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    ### 获得三个维度的相似度特征
                    M = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    C = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    D = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
                    # M, C, D = getSimFeature(folder=folder, signal=11)
                    # M = torch.FloatTensor(M)
                    # C = torch.FloatTensor(C)
                    # D = torch.FloatTensor(D)
                    # inputSize= (M.shape[1]+C.shape[1]+D.shape[1])
                    # print(inputSize)
                    
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)
                    norm = 'tanh'    
                    ## training 
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_val, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_tr, mean, std, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, feat_filt = normalize_Deepsynergy(X_val, mean, std, feat_filt=feat_filt, norm=norm)

                    #print(X_tr.shape,X_val.shape)
                    ## testing    
                    if norm == "tanh_norm":
                        X_train, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_test, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_train, mean, std, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, feat_filt = normalize_Deepsynergy(X_test, mean, std, feat_filt=feat_filt, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                 
                    inputSize = X_tr.shape[1]
                    Neural_Model = DeepSynergy_new(shape, rank, inputSize, 
                                               X_tr,X_val,X_train,X_test,
                                               act_func=nn.ReLU(),dropout=0.5, input_dropout=0.2,
                                               dims=[8182, 4096, 1]).to(device)
                    # [8182, 4096, 1];[4096, 2048, 1]
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    Neural_Model.apply(he_init)
                    # optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)
                    optimizer = optim.SGD(Neural_Model.parameters(), lr=lr, momentum=0.5)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                        # label_pre=outputs
                        # label_pre[ label_pre < threshold ] = 0
                        # label_pre[ label_pre >= threshold ] = 1
                        # outputs[ outputs < 30 ] = 0
                        # outputs[ outputs >= 30 ] = 1
                        # labels_test [labels_test<30] = 0
                        # labels_test [labels_test >=30] =1
                        #print(outputs,labels_test)

                elif self.msi == 80:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    fname = os.path.join('./cpwopt_57R/output/', 'embM_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    M = emb.values
                    #M = torch.FloatTensor(M).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embC_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    C = emb.values
                    #C = torch.FloatTensor(C).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embD_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    D = emb.values
                    #D = torch.FloatTensor(D).to(device)
                    print(M.shape,C.shape,D.shape)

                    M = torch.FloatTensor(M)
                    C = torch.FloatTensor(C)
                    D = torch.FloatTensor(D)
                    ### 得到数据集的特征
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    ### 数据特征标准化
                    norm = 'tanh'    
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2 = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2 = normalize_DTF(X_val, mean, std, mean2, std2,  norm=norm)
                        X_test, mean, std, mean2, std2 = normalize_DTF(X_test, mean, std, mean2, std2, norm=norm)    
                    else:
                        X_tr, mean, std = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std = normalize_DTF(X_val, mean, std, norm=norm)
                        X_test, mean, std = normalize_DTF(X_test, mean, std, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    print(labels_train1,labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    ## 3000
                    #inputSize = rank * len(shape)
                    inputSize = X_tr.shape[1]
                    Neural_Model = DTF_new(shape, rank, inputSize, embeds=[M, C, D], nn_struc=[2048, 1024, 512],
                                       input_dp=0.2, first_dp=0.5, second_dp=0.5).to(device)
                    print(Neural_Model)
                    Neural_Model.apply(he_init)
                    criterion = nn.BCELoss()  # binary crossentropy loss
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)
                    
                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            #print(inputs[-1])
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    #labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                
                elif self.msi == 90:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset

                    # tl.set_backend('numpy')
                    # train_tensor = np.array(self.drug_drug_data.X, copy=True)
                    # trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                    # train_tensor[trainpos_index] = 0
                    # S1 = np.mat(self.drug_drug_data.S1)
                    # S2 = np.mat(self.drug_drug_data.S2)
                    # _, M, C, D = self.numpyModel()(train_tensor, S1, S2,
                    #                                r=self.parameters['r'],
                    #                                mu=self.parameters['mu'], eta=self.parameters['eta'],
                    #                                alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                    #                                lam=self.parameters['lam'],
                    #                                tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                    #                                )
                    # print('CTF')
                    # # print(M.shape, C.shape, D.shape)
                    # M = torch.FloatTensor(M).to(device)
                    # C = torch.FloatTensor(C).to(device)
                    # D = torch.FloatTensor(D).to(device)
                    # print(M.shape, C.shape, D.shape)
                    # print(M)

                    ### 直接导入提前学习好的因子矩阵 
                    fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    #fname='/mnt/sda/liupei/NCTF/newCode/hmddv32/compare/neg/CTF_embeds/1n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
                        M, C, D = pickle.load(f)
                    
                    M = torch.FloatTensor(M).to(device)
                    C = torch.FloatTensor(C).to(device)
                    D = torch.FloatTensor(D).to(device)
                    
                    Neural_Model = CTF_DDI(shape, rank, hids_size=[256, 256, 128], embeds=[M, C, D], device=device).to(device)
                    print(Neural_Model)
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([testModel,idxs_test,labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                # print(idxs_test.T)
                # print(idxs_test.T[0],len(idxs_test.T[0]))
                results = pd.DataFrame({
                    'time': [i] * len(idxs_test.T[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test.T[0]),  # 假设这是第 1 次
                    'm1': idxs_test.T[0],
                    'm2': idxs_test.T[1],
                    'd': idxs_test.T[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': outputs.T[0].cpu().numpy()  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)
                
                metrics = self.get_metrics_1(labels_test.cpu().numpy(), outputs.T[0].cpu().numpy())
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, mname, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j = j + 1

            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t', i + 1, ':\t', result)
            avgmetrics_tensor_10 = avgmetrics_tensor_10 + result

        fname = os.path.join('compareTF', mname + '_hmddv3.2_'+str(self.negs)+'neg_results_0.5lam_fixed.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        #print(j)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t', results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t', results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score = np.mat(real_score)
        predict_score = np.mat(predict_score)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]

        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1

        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN

        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T

        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])

        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])

        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)

        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]

        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision


def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'


if __name__ == '__main__':
    fix_seed(2024)
    ### 导入数据
    since = time.time()
    #df = pd.DataFrame(columns=['methods', 'times', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    methodList = [20, 70, 80, 90]
    mnameList = ['Costco', 'DeepSynergy','DTF', 'CTF_DDI']
    ## 设置参数
    lr = 0.0001  ## 设置均不同
    batch_size = 1024
    epoch = 500
    shape = 3
    r = 57
    nc = int(2*r)
    times = 5
    folds = 5
    #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    #signal = 11
    miRNA_num=351
    disease_num=325
    #drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
    mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    i = 0
    for msi in [20]:
        mname = mnameList[methodList.index(msi)]
        print(mname, msi)
        #for neg in [1,2,4,6,8,10]:
        for neg in [1]:
            folder = '/mnt/sda/liupei/NCTF_new/data/hmddv32_neg/'+str(neg)+'n'
            signal = 11
            print(neg)
            drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num,filefolder=folder,signal=signal,neg=neg)
            since1 = time.time()
            if msi == 20:
                ## Costco
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',msi=msi, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=500, batch_size=256, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 70:
                ### DeepSynergy DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=70, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=64, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
    
            elif msi == 80:
                ## DTF batch_size=128 r=1000
                ### DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=80, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=128, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 90:
                ## CTF_DDI batch_size=1000 epoch=300 r=51 max_iter=200
                signal = 21  # 22
                drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal,neg=neg)
                # DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='CTF',
                                         msi=90, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=300, batch_size=1000, nc=nc,
                                         r=r, mu=0.5, eta=0.2, alpha=0.5, beta=0.5,lam=0.5, tol=1e-6, max_iter=100)

            aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
            df.loc[i] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
            # experiment.CV_triplet()[0]
            print(f"i={i}\ttimes={times}\tmethods={mname}\tmsi={msi}\tneg={neg}")
            print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
            i = i + 1
            time_elapsed1 = time.time() - since1
            print(time_elapsed1 // 60, time_elapsed1 % 60)
            
    df.to_csv('Costco_negResults_256.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)

Costco 20
1
(351, 351)
(325, 325)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
Costco
Costco(
  (embeds): ModuleList(
    (0-1): 2 x Embedding(351, 57)
    (2): Embedding(325, 57)
  )
  (conv1): Conv2d(1, 114, kernel_size=(57, 1), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(1, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=114, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=1, bias=True)
)


 16%|███████████████████▎                                                                                                   | 81/500 [00:27<02:20,  2.98it/s]


Fold 1/5, Test Loss: 0.070948
tensor([[0.8420],
        [0.7804],
        [0.6991],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/Costco_0_times_0_foldscores.pkl
Costco
Costco(
  (embeds): ModuleList(
    (0-1): 2 x Embedding(351, 57)
    (2): Embedding(325, 57)
  )
  (conv1): Conv2d(1, 114, kernel_size=(57, 1), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(1, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=114, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=1, bias=True)
)


 23%|███████████████████████████▌                                                                                          | 117/500 [00:37<02:01,  3.15it/s]


Fold 2/5, Test Loss: 0.065844
tensor([[0.0000],
        [0.2586],
        [0.8195],
        ...,
        [0.3849],
        [0.0000],
        [0.0000]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/Costco_0_times_1_foldscores.pkl
Costco
Costco(
  (embeds): ModuleList(
    (0-1): 2 x Embedding(351, 57)
    (2): Embedding(325, 57)
  )
  (conv1): Conv2d(1, 114, kernel_size=(57, 1), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(1, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=114, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=1, bias=True)
)


...

Fold 3/5, Test Loss: 0.067112
tensor([[0.8788],
        [0.9161],
        [0.0141],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/Costco_4_times_2_foldscores.pkl
Costco
Costco(
  (embeds): ModuleList(
    (0-1): 2 x Embedding(351, 57)
    (2): Embedding(325, 57)
  )
  (conv1): Conv2d(1, 114, kernel_size=(57, 1), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(1, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=114, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=1, bias=True)
)


 21%|████████████████████████▎                                                                                             | 103/500 [00:32<02:07,  3.12it/s]


Fold 4/5, Test Loss: 0.063449
tensor([[0.9658],
        [0.8647],
        [0.9949],
        ...,
        [0.0000],
        [0.0000],
        [0.3722]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/Costco_4_times_3_foldscores.pkl
Costco
Costco(
  (embeds): ModuleList(
    (0-1): 2 x Embedding(351, 57)
    (2): Embedding(325, 57)
  )
  (conv1): Conv2d(1, 114, kernel_size=(57, 1), stride=(1, 1))
  (conv2): Conv2d(114, 114, kernel_size=(1, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=114, out_features=114, bias=True)
  (fc2): Linear(in_features=114, out_features=1, bias=True)
)


 23%|███████████████████████████▌                                                                                          | 117/500 [00:38<02:06,  3.02it/s]

Fold 5/5, Test Loss: 0.064912
tensor([[0.8991],
        [0.8527],
        [1.0243],
        ...,
        [0.0000],
        [0.0000],
        [0.0000]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/Costco_4_times_4_foldscores.pkl
Times:	 5 :	 [[0.9569 0.9656 0.92   0.9195 0.9262 0.9129 0.9142]]
final:	 [[0.9463 0.9456 0.9077 0.9001 0.9311 0.8692 0.8914]]
i=0	times=5	methods=Costco	msi=20	neg=1
auc=0.9456	aupr=0.9463	f1=0.9077	acc=0.9001	recall=0.9311	spe=0.8692	pre=0.8914

12.0 10.528818845748901
12.0 10.723098278045654


In [4]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

from DataCombine_neg import GetData
from ourMethod_gpu import Model
from newNCTF_NN_2 import NCTF_ConvKAN_16, NCTF_ConvKAN_16_noeca1, NCTF_ConvKAN_16_noeca2, NCTF_ConvKAN_16_noeca12
from newNCTF_NN_2 import NCTF_ConvMLP_16
from newNCTF_NN_2 import NCTF_ConvKAN_13, NCTF_ConvKAN_1, ConvKAN
from newNCTF_NN_2 import NCTF_ConvKAN_17, NCTF_ConvKAN_11, ConvKAN_1
from newNCTF_NN_2 import Costco
from newNCTF_NN_2 import CTF_DDI
from newNCTF_NN_2 import DeepSynergy_new, DTF_new
from compareNumpyMethod import Model as numpyModel
#from DataCombine_drug import GetData
import pandas as pd
import os
from torch.backends import cudnn
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
import math
import time
import tensorly as tl
from tqdm import tqdm
import gzip


# from utils import draw

# tl.set_backend('pytorch')

def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

def normalize_Deepsynergy(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt)      

def normalize_DTF(X, means1=None, std1=None, means2=None, std2=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    X = np.ascontiguousarray(X)
    if norm is None:
        return (X, means1, std1, feat_filt)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1
    if norm == 'norm':
        return(X, means1, std1)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        return(X, means1, std1, means2, std2)
        
class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', msi=10, times=10, negs=1,
                 lr=0.001, epoch=150, batch_size=2048, nc=57,
                 **kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.numpyModel = numpyModel(model_name)
        self.msi = msi
        self.times = times
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.channel = nc
        self.negs = negs
        self.parameters = kwargs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = 5
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'methods', 'times', 'folds', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity','precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))

            for k in range(k_folds):
                ### Train data
                # posIndex_train = torch.tensor(torch.nonzero(train_X == 1), dtype=torch.int)
                posIndex_train = torch.tensor(index_matrix[:, np.where(poscv != k)[0]]).T
                negIndex_train = torch.tensor(neg_matrix[:, np.where(negcv != k)[0]]).T
                idxs_train = torch.cat((posIndex_train, negIndex_train), dim=0)
                # print(idxs_train)
                # print(idxs_train.shape)
                poslabel_train = torch.ones(posIndex_train.shape[0])
                neglabel_train = torch.zeros(negIndex_train.shape[0])
                labels_train = torch.cat((poslabel_train, neglabel_train), dim=0)
                # print(labels_train.shape)

                ### 划分验证集
                idxs = idxs_train.numpy().astype(int)
                vals = labels_train.numpy().astype(float)
                # print(idxs,vals)
                # print(idxs.shape, vals.shape)
                idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                # idxs_train, idxs_val, labels_train, labels_val = train_test_split(idxs_train, labels_train, test_size=0.1)

                ### Test data
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = torch.cat((posIndex_test, negIndex_test), dim=0)
                # print(idxs_test)
                # print(idxs_test.shape)
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                # print(labels_test.shape)

                # 模型超参数
                shape = self.drug_drug_data.X.shape
                rank = self.parameters['r']
                nc = self.channel
                lr = self.lr
                epochs = self.epoch
                batch_size = self.batch_size
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                methodList = [20, 70, 80, 90]
                mnameList = ['Costco', 'DeepSynergy', 'DTF', 'CTF_DDI']

                ### 构建深度非线性模型 ### 定义损失函数 和 优化器
                if self.msi == 20:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)

                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    Neural_Model = Costco(shape, rank, nc, device).to(device)  # Costco
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                elif self.msi == 70:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    ### 获得三个维度的相似度特征
                    M = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    C = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    D = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
                    # M, C, D = getSimFeature(folder=folder, signal=11)
                    # M = torch.FloatTensor(M)
                    # C = torch.FloatTensor(C)
                    # D = torch.FloatTensor(D)
                    # inputSize= (M.shape[1]+C.shape[1]+D.shape[1])
                    # print(inputSize)
                    
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)
                    norm = 'tanh'    
                    ## training 
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_val, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_tr, mean, std, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, feat_filt = normalize_Deepsynergy(X_val, mean, std, feat_filt=feat_filt, norm=norm)

                    #print(X_tr.shape,X_val.shape)
                    ## testing    
                    if norm == "tanh_norm":
                        X_train, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_test, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_train, mean, std, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, feat_filt = normalize_Deepsynergy(X_test, mean, std, feat_filt=feat_filt, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                 
                    inputSize = X_tr.shape[1]
                    Neural_Model = DeepSynergy_new(shape, rank, inputSize, 
                                               X_tr,X_val,X_train,X_test,
                                               act_func=nn.ReLU(),dropout=0.5, input_dropout=0.2,
                                               dims=[8182, 4096, 1]).to(device)
                    # [8182, 4096, 1];[4096, 2048, 1]
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    Neural_Model.apply(he_init)
                    # optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)
                    optimizer = optim.SGD(Neural_Model.parameters(), lr=lr, momentum=0.5)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                        # label_pre=outputs
                        # label_pre[ label_pre < threshold ] = 0
                        # label_pre[ label_pre >= threshold ] = 1
                        # outputs[ outputs < 30 ] = 0
                        # outputs[ outputs >= 30 ] = 1
                        # labels_test [labels_test<30] = 0
                        # labels_test [labels_test >=30] =1
                        #print(outputs,labels_test)

                elif self.msi == 80:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    fname = os.path.join('./cpwopt_57R/output/', 'embM_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    M = emb.values
                    #M = torch.FloatTensor(M).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embC_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    C = emb.values
                    #C = torch.FloatTensor(C).to(device)
                    fname = os.path.join('./cpwopt_57R/output/', 'embD_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    D = emb.values
                    #D = torch.FloatTensor(D).to(device)
                    print(M.shape,C.shape,D.shape)

                    M = torch.FloatTensor(M)
                    C = torch.FloatTensor(C)
                    D = torch.FloatTensor(D)
                    ### 得到数据集的特征
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    ### 数据特征标准化
                    norm = 'tanh'    
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2 = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2 = normalize_DTF(X_val, mean, std, mean2, std2,  norm=norm)
                        X_test, mean, std, mean2, std2 = normalize_DTF(X_test, mean, std, mean2, std2, norm=norm)    
                    else:
                        X_tr, mean, std = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std = normalize_DTF(X_val, mean, std, norm=norm)
                        X_test, mean, std = normalize_DTF(X_test, mean, std, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    print(labels_train1,labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    ## 3000
                    #inputSize = rank * len(shape)
                    inputSize = X_tr.shape[1]
                    Neural_Model = DTF_new(shape, rank, inputSize, embeds=[M, C, D], nn_struc=[2048, 1024, 512],
                                       input_dp=0.2, first_dp=0.5, second_dp=0.5).to(device)
                    print(Neural_Model)
                    Neural_Model.apply(he_init)
                    criterion = nn.BCELoss()  # binary crossentropy loss
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)
                    
                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            #print(inputs[-1])
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    #labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                
                elif self.msi == 90:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset

                    # tl.set_backend('numpy')
                    # train_tensor = np.array(self.drug_drug_data.X, copy=True)
                    # trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                    # train_tensor[trainpos_index] = 0
                    # S1 = np.mat(self.drug_drug_data.S1)
                    # S2 = np.mat(self.drug_drug_data.S2)
                    # _, M, C, D = self.numpyModel()(train_tensor, S1, S2,
                    #                                r=self.parameters['r'],
                    #                                mu=self.parameters['mu'], eta=self.parameters['eta'],
                    #                                alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                    #                                lam=self.parameters['lam'],
                    #                                tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                    #                                )
                    # print('CTF')
                    # # print(M.shape, C.shape, D.shape)
                    # M = torch.FloatTensor(M).to(device)
                    # C = torch.FloatTensor(C).to(device)
                    # D = torch.FloatTensor(D).to(device)
                    # print(M.shape, C.shape, D.shape)
                    # print(M)

                    ### 直接导入提前学习好的因子矩阵 
                    fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    #fname='/mnt/sda/liupei/NCTF/newCode/hmddv32/compare/neg/CTF_embeds/1n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
                        M, C, D = pickle.load(f)
                    
                    M = torch.FloatTensor(M).to(device)
                    C = torch.FloatTensor(C).to(device)
                    D = torch.FloatTensor(D).to(device)
                    
                    Neural_Model = CTF_DDI(shape, rank, hids_size=[256, 256, 128], embeds=[M, C, D], device=device).to(device)
                    print(Neural_Model)
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([testModel,idxs_test,labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                # print(idxs_test.T)
                # print(idxs_test.T[0],len(idxs_test.T[0]))
                results = pd.DataFrame({
                    'time': [i] * len(idxs_test.T[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test.T[0]),  # 假设这是第 1 次
                    'm1': idxs_test.T[0],
                    'm2': idxs_test.T[1],
                    'd': idxs_test.T[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': outputs.T[0].cpu().numpy()  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)
                
                metrics = self.get_metrics_1(labels_test.cpu().numpy(), outputs.T[0].cpu().numpy())
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, mname, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j = j + 1

            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t', i + 1, ':\t', result)
            avgmetrics_tensor_10 = avgmetrics_tensor_10 + result

        fname = os.path.join('compareTF', mname + '_hmddv3.2_'+str(self.negs)+'neg_results_0.5lam_fixed.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        #print(j)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t', results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t', results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score = np.mat(real_score)
        predict_score = np.mat(predict_score)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]

        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1

        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN

        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T

        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])

        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])

        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)

        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]

        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision


def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'


if __name__ == '__main__':
    fix_seed(2024)
    ### 导入数据
    since = time.time()
    #df = pd.DataFrame(columns=['methods', 'times', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    methodList = [20, 70, 80, 90]
    mnameList = ['Costco', 'DeepSynergy','DTF', 'CTF_DDI']
    ## 设置参数
    lr = 0.0001  ## 设置均不同
    #batch_size = 256
    epoch = 500
    shape = 3
    r = 57
    nc = int(2*r)
    times = 5
    folds = 5
    #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    #signal = 11
    miRNA_num=351
    disease_num=325
    #drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
    mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    i = 0
    for msi in [70]:
        mname = mnameList[methodList.index(msi)]
        print(mname, msi)
        #for neg in [1,2,4,6,8,10]:
        for neg in [1]:
            folder = '/mnt/sda/liupei/NCTF_new/data/hmddv32_neg/'+str(neg)+'n'
            signal = 11
            print(neg)
            drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num,filefolder=folder,signal=signal,neg=neg)
            since1 = time.time()
            if msi == 20:
                ## Costco
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',msi=msi, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=500, batch_size=256, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 70:
                ### DeepSynergy DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=70, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=64, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
    
            elif msi == 80:
                ## DTF batch_size=128 r=1000
                ### DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=80, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=128, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 90:
                ## CTF_DDI batch_size=1000 epoch=300 r=51 max_iter=200
                signal = 21  # 22
                drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal,neg=neg)
                # DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='CTF',
                                         msi=90, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=300, batch_size=1000, nc=nc,
                                         r=r, mu=0.5, eta=0.2, alpha=0.5, beta=0.5,lam=0.5, tol=1e-6, max_iter=100)

            aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
            df.loc[i] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
            # experiment.CV_triplet()[0]
            print(f"i={i}\ttimes={times}\tmethods={mname}\tmsi={msi}\tneg={neg}")
            print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
            i = i + 1
            time_elapsed1 = time.time() - since1
            print(time_elapsed1 // 60, time_elapsed1 % 60)
            
    df.to_csv('DeepSynergy_negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)

DeepSynergy 70
1
(351, 351)
(325, 325)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
DeepSynergy
(21137, 3)
torch.Size([21137, 1027])
(2349, 3)
torch.Size([2349, 1027])
torch.Size([23486, 3])
torch.Size([23486, 1027])
torch.Size([5872, 3])
torch.Size([5872, 1027])
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
DeepSynergy_new(
  (layers): ModuleList(
    (0): Linear(in_features=1027, out_features=8182, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=8182, out_features=4096, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=4096, out_features=1, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


 15%|█████████████████▏                                                                                                   | 147/1000 [03:45<21:46,  1.53s/it]


Fold 1/5, Test Loss: 0.072138
tensor([[ 0.9746],
        [ 0.7280],
        [ 0.8086],
        ...,
        [ 0.0974],
        [-0.0368],
        [-0.4216]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DeepSynergy_0_times_0_foldscores.pkl
DeepSynergy
(21137, 3)
torch.Size([21137, 1027])
(2349, 3)
torch.Size([2349, 1027])
torch.Size([23486, 3])
torch.Size([23486, 1027])
torch.Size([5872, 3])
torch.Size([5872, 1027])
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
DeepSynergy_new(
  (layers): ModuleList(
    (0): Linear(in_features=1027, out_features=8182, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=8182, out_features=4096, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=4096, out_features=1, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


 19%|█████████████████████▊                                                                                               | 186/1000 [04:43<20:41,  1.53s/it]


Fold 2/5, Test Loss: 0.072162
tensor([[ 0.5657],
        [ 0.6240],
        [ 0.6703],
        ...,
        [ 0.5988],
        [-0.2978],
        [-0.1317]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DeepSynergy_0_times_1_foldscores.pkl
DeepSynergy
(21137, 3)
torch.Size([21137, 1027])
(2349, 3)
torch.Size([2349, 1027])
torch.Size([23486, 3])
torch.Size([23486, 1027])
torch.Size([5872, 3])
torch.Size([5872, 1027])
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
DeepSynergy_new(
  (layers): ModuleList(
    (0): Linear(in_features=1027, out_features=8182, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=8182, out_features=4096, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=4096, out_features=1, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


...

Fold 3/5, Test Loss: 0.070167
tensor([[ 0.4989],
        [ 0.6365],
        [ 0.5166],
        ...,
        [ 0.0115],
        [-0.0994],
        [ 0.0376]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DeepSynergy_4_times_2_foldscores.pkl
DeepSynergy
(21137, 3)
torch.Size([21137, 1027])
(2349, 3)
torch.Size([2349, 1027])
torch.Size([23486, 3])
torch.Size([23486, 1027])
torch.Size([5872, 3])
torch.Size([5872, 1027])
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
(21137, 1027) (2349, 1027) (23486, 1027) (5872, 1027)
DeepSynergy_new(
  (layers): ModuleList(
    (0): Linear(in_features=1027, out_features=8182, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=8182, out_features=4096, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=4096, out_features=1, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


 16%|██████████████████▉                                                                                                  | 162/1000 [06:50<35:23,  2.53s/it]


Fold 4/5, Test Loss: 0.072650
tensor([[0.7013],
        [0.9574],
        [0.7751],
        ...,
        [0.0913],
        [0.1622],
        [0.1239]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DeepSynergy_4_times_3_foldscores.pkl
DeepSynergy
(21139, 3)
torch.Size([21139, 1027])
(2349, 3)
torch.Size([2349, 1027])
torch.Size([23488, 3])
torch.Size([23488, 1027])
torch.Size([5870, 3])
torch.Size([5870, 1027])
(21139, 1027) (2349, 1027) (23488, 1027) (5870, 1027)
(21139, 1027) (2349, 1027) (23488, 1027) (5870, 1027)
DeepSynergy_new(
  (layers): ModuleList(
    (0): Linear(in_features=1027, out_features=8182, bias=True)
    (1): Dropout(p=0.2, inplace=False)
    (2): Linear(in_features=8182, out_features=4096, bias=True)
    (3): ReLU()
    (4): Dropout(p=0.5, inplace=False)
    (5): Linear(in_features=4096, out_features=1, bias=True)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


 16%|██████████████████▏                                                                                                  | 155/1000 [06:49<37:10,  2.64s/it]


Fold 5/5, Test Loss: 0.071830
tensor([[0.5092],
        [0.8848],
        [0.8394],
        ...,
        [0.2347],
        [0.0758],
        [0.1849]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DeepSynergy_4_times_4_foldscores.pkl
Times:	 5 :	 [[0.9796 0.9788 0.928  0.928  0.9289 0.9271 0.9274]]
final:	 [[0.9788 0.9781 0.9274 0.9272 0.929  0.9255 0.9259]]
i=0	times=5	methods=DeepSynergy	msi=70	neg=1
auc=0.9781	aupr=0.9788	f1=0.9274	acc=0.9272	recall=0.929	spe=0.9255	pre=0.9259

127.0 23.297795295715332
127.0 23.490610361099243


In [5]:
import sys
import os
# 添加模块所在的文件夹到 sys.path
folder_path = "/mnt/sda/liupei/NCTF_new/src/"
sys.path.append(folder_path)

from DataCombine_neg import GetData
from ourMethod_gpu import Model
from newNCTF_NN_2 import NCTF_ConvKAN_16, NCTF_ConvKAN_16_noeca1, NCTF_ConvKAN_16_noeca2, NCTF_ConvKAN_16_noeca12
from newNCTF_NN_2 import NCTF_ConvMLP_16
from newNCTF_NN_2 import NCTF_ConvKAN_13, NCTF_ConvKAN_1, ConvKAN
from newNCTF_NN_2 import NCTF_ConvKAN_17, NCTF_ConvKAN_11, ConvKAN_1
from newNCTF_NN_2 import Costco
from newNCTF_NN_2 import CTF_DDI
from newNCTF_NN_2 import DeepSynergy_new, DTF_new
from compareNumpyMethod import Model as numpyModel
#from DataCombine_drug import GetData
import pandas as pd
import os
from torch.backends import cudnn
import random
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, f1_score, precision_recall_curve, average_precision_score
import math
import time
import tensorly as tl
from tqdm import tqdm
import gzip


# from utils import draw

# tl.set_backend('pytorch')

def he_init(m):
    if isinstance(m, nn.Linear):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')

def normalize_Deepsynergy(X, means1=None, std1=None, means2=None, std2=None, feat_filt=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    if feat_filt is None:
        feat_filt = std1!=0
    X = X[:,feat_filt]
    X = np.ascontiguousarray(X)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1[feat_filt]
    if norm == 'norm':
        return(X, means1, std1, feat_filt)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1, feat_filt)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        X[:,std2==0]=0
        return(X, means1, std1, means2, std2, feat_filt)      

def normalize_DTF(X, means1=None, std1=None, means2=None, std2=None, norm='tanh_norm'):
    if std1 is None:
        std1 = np.nanstd(X, axis=0)
    X = np.ascontiguousarray(X)
    if norm is None:
        return (X, means1, std1, feat_filt)
    if means1 is None:
        means1 = np.mean(X, axis=0)
    X = (X-means1)/std1
    if norm == 'norm':
        return(X, means1, std1)
    elif norm == 'tanh':
        return(np.tanh(X), means1, std1)
    elif norm == 'tanh_norm':
        X = np.tanh(X)
        if means2 is None:
            means2 = np.mean(X, axis=0)
        if std2 is None:
            std2 = np.std(X, axis=0)
        X = (X-means2)/std2
        return(X, means1, std1, means2, std2)
        
class Experiments(object):

    def __init__(self, drug_drug_data, model_name='NCTF', msi=10, times=10, negs=1,
                 lr=0.001, epoch=150, batch_size=2048, nc=57,
                 **kwargs):
        super().__init__()
        self.drug_drug_data = drug_drug_data
        self.model = Model(model_name)
        self.numpyModel = numpyModel(model_name)
        self.msi = msi
        self.times = times
        self.lr = lr
        self.epoch = epoch
        self.batch_size = batch_size
        self.channel = nc
        self.negs = negs
        self.parameters = kwargs

    def CV_triplet(self):
        fix_seed(2024)
        k_folds = 5
        np.random.seed(2024)
        metrics_tensor_all = np.zeros((1, 7))
        avgmetrics_tensor_10 = np.zeros((1, 7))
        j = 0
        df = pd.DataFrame(columns=['j', 'methods', 'times', 'folds', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity','precision'])
        for i in range(self.times):
            index_matrix = self.drug_drug_data.posidx[i].numpy().T
            poscv = self.drug_drug_data.poscv[i].numpy()
            neg_matrix = self.drug_drug_data.negidx[i].numpy().T
            negcv = self.drug_drug_data.negcv[i].numpy()
            metrics_tensor = np.zeros((1, 7))

            for k in range(k_folds):
                ### Train data
                # posIndex_train = torch.tensor(torch.nonzero(train_X == 1), dtype=torch.int)
                posIndex_train = torch.tensor(index_matrix[:, np.where(poscv != k)[0]]).T
                negIndex_train = torch.tensor(neg_matrix[:, np.where(negcv != k)[0]]).T
                idxs_train = torch.cat((posIndex_train, negIndex_train), dim=0)
                # print(idxs_train)
                # print(idxs_train.shape)
                poslabel_train = torch.ones(posIndex_train.shape[0])
                neglabel_train = torch.zeros(negIndex_train.shape[0])
                labels_train = torch.cat((poslabel_train, neglabel_train), dim=0)
                # print(labels_train.shape)

                ### 划分验证集
                idxs = idxs_train.numpy().astype(int)
                vals = labels_train.numpy().astype(float)
                # print(idxs,vals)
                # print(idxs.shape, vals.shape)
                idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                # idxs_train, idxs_val, labels_train, labels_val = train_test_split(idxs_train, labels_train, test_size=0.1)

                ### Test data
                posIndex_test = torch.tensor(index_matrix[:, np.where(poscv == k)[0]], dtype=torch.int).T
                negIndex_test = torch.tensor(neg_matrix[:, np.where(negcv == k)[0]], dtype=torch.int).T
                idxs_test = torch.cat((posIndex_test, negIndex_test), dim=0)
                # print(idxs_test)
                # print(idxs_test.shape)
                poslabel_test = torch.ones(posIndex_test.shape[0])
                neglabel_test = torch.zeros(negIndex_test.shape[0])
                labels_test = torch.cat((poslabel_test, neglabel_test), dim=0)
                # print(labels_test.shape)

                # 模型超参数
                shape = self.drug_drug_data.X.shape
                rank = self.parameters['r']
                nc = self.channel
                lr = self.lr
                epochs = self.epoch
                batch_size = self.batch_size
                device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

                methodList = [20, 70, 80, 90]
                mnameList = ['Costco', 'DeepSynergy', 'DTF', 'CTF_DDI']

                ### 构建深度非线性模型 ### 定义损失函数 和 优化器
                if self.msi == 20:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)

                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    Neural_Model = Costco(shape, rank, nc, device).to(device)  # Costco
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                elif self.msi == 70:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    ### 获得三个维度的相似度特征
                    M = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    C = torch.tensor(self.drug_drug_data.S1, dtype=torch.float32)
                    D = torch.tensor(self.drug_drug_data.S2, dtype=torch.float32)
                    # folder = '/mnt/sda/liupei/NCTF/newCode/data/hmddv32_neg/'+str(neg)+'n'
                    # M, C, D = getSimFeature(folder=folder, signal=11)
                    # M = torch.FloatTensor(M)
                    # C = torch.FloatTensor(C)
                    # D = torch.FloatTensor(D)
                    # inputSize= (M.shape[1]+C.shape[1]+D.shape[1])
                    # print(inputSize)
                    
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    print(f123.shape)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)
                    norm = 'tanh'    
                    ## training 
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_val, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_tr, mean, std, feat_filt = normalize_Deepsynergy(X_tr, norm=norm)
                        X_val, mean, std, feat_filt = normalize_Deepsynergy(X_val, mean, std, feat_filt=feat_filt, norm=norm)

                    #print(X_tr.shape,X_val.shape)
                    ## testing    
                    if norm == "tanh_norm":
                        X_train, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, mean2, std2, feat_filt = normalize_Deepsynergy(X_test, mean, std, mean2, std2, 
                                                                              feat_filt=feat_filt, norm=norm)
                    else:
                        X_train, mean, std, feat_filt = normalize_Deepsynergy(X_train, norm=norm)
                        X_test, mean, std, feat_filt = normalize_Deepsynergy(X_test, mean, std, feat_filt=feat_filt, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                 
                    inputSize = X_tr.shape[1]
                    Neural_Model = DeepSynergy_new(shape, rank, inputSize, 
                                               X_tr,X_val,X_train,X_test,
                                               act_func=nn.ReLU(),dropout=0.5, input_dropout=0.2,
                                               dims=[8182, 4096, 1]).to(device)
                    # [8182, 4096, 1];[4096, 2048, 1]
                    print(Neural_Model)
                    criterion = nn.MSELoss()
                    Neural_Model.apply(he_init)
                    # optimizer = optim.Adam(Neural_Model.parameters(), lr=lr)
                    optimizer = optim.SGD(Neural_Model.parameters(), lr=lr, momentum=0.5)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                        # label_pre=outputs
                        # label_pre[ label_pre < threshold ] = 0
                        # label_pre[ label_pre >= threshold ] = 1
                        # outputs[ outputs < 30 ] = 0
                        # outputs[ outputs >= 30 ] = 1
                        # labels_test [labels_test<30] = 0
                        # labels_test [labels_test >=30] =1
                        #print(outputs,labels_test)

                elif self.msi == 80:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    fname = os.path.join('/mnt/sda/liupei/NCTF_new/hmddv32/compare/neg/cpwopt_57R/'+str(self.negs)+'n/output/'
                                         'embM_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    M = emb.values
                    #M = torch.FloatTensor(M).to(device)
                    fname = os.path.join('/mnt/sda/liupei/NCTF_new/hmddv32/compare/neg/cpwopt_57R/'+str(self.negs)+'n/output/'
                                         'embC_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    C = emb.values
                    #C = torch.FloatTensor(C).to(device)
                    fname = os.path.join('/mnt/sda/liupei/NCTF_new/hmddv32/compare/neg/cpwopt_57R/'+str(self.negs)+'n/output/'
                                         'embD_{}_times_{}_fold.csv'.format(i + 1, k + 1))
                    emb = pd.read_csv(fname, header=None)
                    D = emb.values
                    #D = torch.FloatTensor(D).to(device)
                    print(M.shape,C.shape,D.shape)

                    M = torch.FloatTensor(M)
                    C = torch.FloatTensor(C)
                    D = torch.FloatTensor(D)
                    ### 得到数据集的特征
                    print(idxs_train1.shape)
                    f_1 = M[idxs_train1.T[0]]
                    f_2 = C[idxs_train1.T[1]]
                    f_3 = D[idxs_train1.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_tr=f123.numpy()

                    print(idxs_val.shape)
                    f_1 = M[idxs_val.T[0]]
                    f_2 = C[idxs_val.T[1]]
                    f_3 = D[idxs_val.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_val=f123.numpy()

                    print(idxs_train.shape)
                    f_1 = M[idxs_train.T[0]]
                    f_2 = C[idxs_train.T[1]]
                    f_3 = D[idxs_train.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_train=f123.numpy()

                    print(idxs_test.shape)
                    f_1 = M[idxs_test.T[0]]
                    f_2 = C[idxs_test.T[1]]
                    f_3 = D[idxs_test.T[2]]
                    f123 = torch.cat([f_1,f_2,f_3], dim=1)
                    #print(f123)
                    X_test=f123.numpy()
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    ### 数据特征标准化
                    norm = 'tanh'    
                    if norm == "tanh_norm":
                        X_tr, mean, std, mean2, std2 = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std, mean2, std2 = normalize_DTF(X_val, mean, std, mean2, std2,  norm=norm)
                        X_test, mean, std, mean2, std2 = normalize_DTF(X_test, mean, std, mean2, std2, norm=norm)    
                    else:
                        X_tr, mean, std = normalize_DTF(X_tr, norm=norm)
                        X_val, mean, std = normalize_DTF(X_val, mean, std, norm=norm)
                        X_test, mean, std = normalize_DTF(X_test, mean, std, norm=norm)
                    
                    print(X_tr.shape,X_val.shape,X_train.shape,X_test.shape)

                    # 创建数据加载器
                    X_tr = torch.FloatTensor(X_tr)
                    X_val = torch.FloatTensor(X_val)
                    X_test = torch.FloatTensor(X_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    print(labels_train1,labels_val)
                    # labels_test_Score = torch.FloatTensor(labels_test_Score)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(X_tr, labels_train1)
                    val_dataset = TensorDataset(X_val, labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset
                    
                    ## 3000
                    #inputSize = rank * len(shape)
                    inputSize = X_tr.shape[1]
                    Neural_Model = DTF_new(shape, rank, inputSize, embeds=[M, C, D], nn_struc=[2048, 1024, 512],
                                       input_dp=0.2, first_dp=0.5, second_dp=0.5).to(device)
                    print(Neural_Model)
                    Neural_Model.apply(he_init)
                    criterion = nn.BCELoss()  # binary crossentropy loss
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)
                    
                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            #print(inputs[-1])
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                #inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    #labels_test_Score = torch.FloatTensor(labels_test_Score)
                    #labels_test = torch.FloatTensor(labels_test)
                    #print(labels_test)
                    with torch.no_grad():
                        #inputs_gpu = idxs_test.T.to(device)
                        inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        #loss = criterion(outputs, labels_test_Score.unsqueeze(1).to(device))
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)
                
                elif self.msi == 90:
                    mname = mnameList[methodList.index(self.msi)]
                    print(mname)
                    # ### 划分验证集
                    # idxs = idxs_train.numpy().astype(int)
                    # vals = labels_train.numpy().astype(float)
                    # # print(idxs,vals)
                    # # print(idxs.shape, vals.shape)
                    # idxs_train1, idxs_val, labels_train1, labels_val = train_test_split(idxs, vals, test_size=0.1,random_state=2024)
                    
                    # 创建数据加载器
                    idxs_train1 = torch.LongTensor(idxs_train1)
                    idxs_val = torch.LongTensor(idxs_val)
                    # idxs_test = torch.LongTensor(idxs_test)
                    labels_train1 = torch.FloatTensor(labels_train1)
                    labels_val = torch.FloatTensor(labels_val)
                    # labels_test = torch.FloatTensor(labels_test)
                    
                    train_dataset = TensorDataset(*[idxs_train1[:, i] for i in range(len(shape))], labels_train1)
                    val_dataset = TensorDataset(*[idxs_val[:, i] for i in range(len(shape))], labels_val)
                    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
                    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
                    del train_dataset, val_dataset

                    # tl.set_backend('numpy')
                    # train_tensor = np.array(self.drug_drug_data.X, copy=True)
                    # trainpos_index = tuple(index_matrix[:, np.where(poscv == k)[0]])
                    # train_tensor[trainpos_index] = 0
                    # S1 = np.mat(self.drug_drug_data.S1)
                    # S2 = np.mat(self.drug_drug_data.S2)
                    # _, M, C, D = self.numpyModel()(train_tensor, S1, S2,
                    #                                r=self.parameters['r'],
                    #                                mu=self.parameters['mu'], eta=self.parameters['eta'],
                    #                                alpha=self.parameters['alpha'], beta=self.parameters['beta'],
                    #                                lam=self.parameters['lam'],
                    #                                tol=self.parameters['tol'], max_iter=self.parameters['max_iter']
                    #                                )
                    # print('CTF')
                    # # print(M.shape, C.shape, D.shape)
                    # M = torch.FloatTensor(M).to(device)
                    # C = torch.FloatTensor(C).to(device)
                    # D = torch.FloatTensor(D).to(device)
                    # print(M.shape, C.shape, D.shape)
                    # print(M)

                    ### 直接导入提前学习好的因子矩阵 
                    fname='CTF_embeds/'+str(self.negs)+'n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    #fname='/mnt/sda/liupei/NCTF/newCode/hmddv32/compare/neg/CTF_embeds/1n/factors_'+str(i)+'_times_'+str(k)+'_fold.pkl'
                    with open(fname, 'rb') as f:  # Python 3: open(..., 'rb')
                        M, C, D = pickle.load(f)
                    
                    M = torch.FloatTensor(M).to(device)
                    C = torch.FloatTensor(C).to(device)
                    D = torch.FloatTensor(D).to(device)
                    
                    Neural_Model = CTF_DDI(shape, rank, hids_size=[256, 256, 128], embeds=[M, C, D], device=device).to(device)
                    print(Neural_Model)
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(Neural_Model.parameters(), lr=lr, weight_decay=5e-4)

                    # 训练模型
                    min_val, min_test, min_epoch, final_model = 9999, 9999, 0, 0
                    loss_train_list = []
                    loss_test_list = []
                    for epoch in tqdm(range(epochs)):
                        ##训练
                        Neural_Model.train()
                        train_loss, valid_loss = 0, 0
                        # loss_train_list_batch = []
                        for inputs in train_loader:
                            #print(inputs)
                            optimizer.zero_grad()
                            inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                            #inputs_gpu = inputs[0].to(device)
                            outputs = Neural_Model(inputs_gpu)
                            loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                            loss.backward()
                            optimizer.step()
                            train_loss += loss.item() * len(inputs)
                        train_loss /= len(train_loader.dataset)
                        loss_train_list.append(train_loss)
    
                        # 验证模型
                        Neural_Model.eval()
                        val_loss = 0.0
                        for inputs in val_loader:
                            with torch.no_grad():
                                inputs_gpu = [tensor.to(device) for tensor in inputs[:-1]]
                                #inputs_gpu = inputs[0].to(device)
                                outputs = Neural_Model(inputs_gpu)
                                loss = criterion(outputs, inputs[-1].unsqueeze(1).to(device))
                                val_loss += loss.item() * len(inputs)
                        val_loss /= len(val_loader.dataset)
                        loss_test_list.append(val_loss)
    
                        # if epoch % 5 == 0:
                        #     print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.6f}, Val Loss: {val_loss:.6f}")
    
                        if min_val <= val_loss and epoch - min_epoch >= 10:
                            break
    
                        if min_val > val_loss:
                            min_val = val_loss
                            min_epoch = epoch
                            testModel = Neural_Model
                    
                    testModel.eval()
                    with torch.no_grad():
                        inputs_gpu = idxs_test.T.to(device)
                        #inputs_gpu = X_test.to(device)
                        outputs = testModel(inputs_gpu)
                        loss = criterion(outputs, labels_test.unsqueeze(1).to(device))
                        print(f"Fold {k + 1}/{k_folds}, Test Loss: {loss:.6f}")
                        print(outputs,labels_test)

                ## 存储每折每次的预测和真实值
                fname='pred_score_pkl/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.pkl'
                print(fname)
                with open(fname, 'wb') as f:  # Python 3: open(..., 'wb')
                    pickle.dump([testModel,idxs_test,labels_test.cpu().numpy(),outputs.T[0].cpu().numpy()], f)

                # print(idxs_test.T)
                # print(idxs_test.T[0],len(idxs_test.T[0]))
                results = pd.DataFrame({
                    'time': [i] * len(idxs_test.T[0]),  # 假设这是第 1 折
                    'fold': [k] * len(idxs_test.T[0]),  # 假设这是第 1 次
                    'm1': idxs_test.T[0],
                    'm2': idxs_test.T[1],
                    'd': idxs_test.T[2],
                    'true_label': labels_test.cpu().numpy(),
                    'pred_score': outputs.T[0].cpu().numpy()  # 假设 preds 是一个二维数组，取第二列作为预测概率
                })
                # 保存为 CSV 文件
                fname='pred_score_csv/'+mname+'_'+str(i)+'_times_'+str(k)+'_foldscores.csv'
                results.to_csv(fname, index=False)
                
                metrics = self.get_metrics_1(labels_test.cpu().numpy(), outputs.T[0].cpu().numpy())
                metrics_tensor = metrics_tensor + metrics
                metrics_tensor_all = metrics_tensor_all + metrics
                aupr, auc_value, f1_score, accuracy, recall, specificity, precision = metrics
                df.loc[j] = [j, mname, i, k, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
                j = j + 1

            result = np.around(metrics_tensor / k_folds, decimals=4)
            print('Times:\t', i + 1, ':\t', result)
            avgmetrics_tensor_10 = avgmetrics_tensor_10 + result

        fname = os.path.join('compareTF', mname + '_hmddv3.2_'+str(self.negs)+'neg_results_0.5lam_fixed.csv')
        df.to_csv(fname, index=False)  # index=False 表示不写入行索引
        #print(j)
        results_1 = np.around(metrics_tensor_all / j, decimals=4)
        print('final:\t', results_1)
        # results_2 = np.around(avgmetrics_tensor_10 / self.times, decimals=4)
        # print('final:\t', results_2)
        return results_1

    def get_metrics_1(self, real_score, predict_score):
        real_score = np.mat(real_score)
        predict_score = np.mat(predict_score)
        np.random.seed(2024)
        sorted_predict_score = np.array(sorted(list(set(np.array(predict_score).flatten()))))
        sorted_predict_score_num = len(sorted_predict_score)
        thresholds = sorted_predict_score[
            (np.array([sorted_predict_score_num]) * np.arange(1, 1000) / np.array([1000])).astype(int)]
        thresholds = np.mat(thresholds)
        thresholds_num = thresholds.shape[1]

        predict_score_matrix = np.tile(predict_score, (thresholds_num, 1))
        negative_index = np.where(predict_score_matrix < thresholds.T)
        positive_index = np.where(predict_score_matrix >= thresholds.T)
        predict_score_matrix[negative_index] = 0
        predict_score_matrix[positive_index] = 1

        TP = predict_score_matrix * real_score.T
        FP = predict_score_matrix.sum(axis=1) - TP
        FN = real_score.sum() - TP
        TN = len(real_score.T) - TP - FP - FN

        fpr = FP / (FP + TN)
        tpr = TP / (TP + FN)
        ROC_dot_matrix = np.mat(sorted(np.column_stack((fpr, tpr)).tolist())).T
        # print(ROC_dot_matrix)
        ROC_dot_matrix.T[0] = [0, 0]
        ROC_dot_matrix = np.c_[ROC_dot_matrix, [1, 1]]
        x_ROC = ROC_dot_matrix[0].T
        y_ROC = ROC_dot_matrix[1].T

        auc = 0.5 * (x_ROC[1:] - x_ROC[:-1]).T * (y_ROC[:-1] + y_ROC[1:])

        recall_list = tpr
        precision_list = TP / (TP + FP)
        PR_dot_matrix = np.mat(sorted(np.column_stack((recall_list, -precision_list)).tolist())).T
        PR_dot_matrix[1, :] = -PR_dot_matrix[1, :]
        PR_dot_matrix.T[0] = [0, 1]
        PR_dot_matrix = np.c_[PR_dot_matrix, [1, 0]]
        x_PR = PR_dot_matrix[0].T
        y_PR = PR_dot_matrix[1].T
        aupr = 0.5 * (x_PR[1:] - x_PR[:-1]).T * (y_PR[:-1] + y_PR[1:])

        f1_score_list = 2 * TP / (len(real_score.T) + TP - TN)
        accuracy_list = (TP + TN) / len(real_score.T)
        specificity_list = TN / (TN + FP)

        max_index = np.argmax(f1_score_list)
        f1_score = f1_score_list[max_index, 0]
        accuracy = accuracy_list[max_index, 0]
        specificity = specificity_list[max_index, 0]
        recall = recall_list[max_index, 0]
        precision = precision_list[max_index, 0]

        return aupr[0, 0], auc[0, 0], f1_score, accuracy, recall, specificity, precision


def fix_seed(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    cudnn.deterministic = True
    cudnn.benchmark = False

    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'


if __name__ == '__main__':
    fix_seed(2024)
    ### 导入数据
    since = time.time()
    #df = pd.DataFrame(columns=['methods', 'times', 'aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    methodList = [20, 70, 80, 90]
    mnameList = ['Costco', 'DeepSynergy','DTF', 'CTF_DDI']
    ## 设置参数
    lr = 0.0001  ## 设置均不同
    #batch_size = 256
    epoch = 500
    shape = 3
    r = 57
    nc = int(2*r)
    times = 5
    folds = 5
    #folder = '/mnt/sda/liupei/NCTF/newCode/data/newmmd_10times_5cv'
    #signal = 11
    miRNA_num=351
    disease_num=325
    #drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal)
    mu,eta,alpha,beta,lam=0.75,0.125,0.25,0.25,0.001
    df = pd.DataFrame(columns=['neg','aupr', 'auc', 'f1_score', 'accuracy', 'recall', 'specificity', 'precision'])
    i = 0
    for msi in [80]:
        mname = mnameList[methodList.index(msi)]
        print(mname, msi)
        #for neg in [1,2,4,6,8,10]:
        for neg in [1]:
            folder = '/mnt/sda/liupei/NCTF_new/data/hmddv32_neg/'+str(neg)+'n'
            signal = 11
            print(neg)
            drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num,filefolder=folder,signal=signal,neg=neg)
            since1 = time.time()
            if msi == 20:
                ## Costco
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',msi=msi, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=500, batch_size=256, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 70:
                ### DeepSynergy DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=70, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=64, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
    
            elif msi == 80:
                ## DTF batch_size=128 r=1000
                ### DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='NCTF_torch_gpu_float32',
                                         msi=80, times=times, folds=folds,negs=neg,
                                         lr=0.00001, epoch=1000, batch_size=128, nc=nc,
                                         r=r, mu=mu, eta=eta, alpha=alpha, beta=beta,lam=lam, tol=1e-4, max_iter=100)
            elif msi == 90:
                ## CTF_DDI batch_size=1000 epoch=300 r=51 max_iter=200
                signal = 21  # 22
                drug_drug_data = GetData(miRNA_num=miRNA_num, disease_num=disease_num, filefolder=folder, signal=signal,neg=neg)
                # DTF 论文设置的参数
                experiment = Experiments(drug_drug_data, model_name='CTF',
                                         msi=90, times=times, folds=folds,negs=neg,
                                         lr=0.0001, epoch=300, batch_size=1000, nc=nc,
                                         r=r, mu=0.5, eta=0.2, alpha=0.5, beta=0.5,lam=0.5, tol=1e-6, max_iter=100)

            aupr, auc_value, f1_score, accuracy, recall, specificity, precision = experiment.CV_triplet()[0]
            df.loc[i] = [neg, aupr, auc_value, f1_score, accuracy, recall, specificity, precision]
            # experiment.CV_triplet()[0]
            print(f"i={i}\ttimes={times}\tmethods={mname}\tmsi={msi}\tneg={neg}")
            print(f"auc={auc_value}\taupr={aupr}\tf1={f1_score}\tacc={accuracy}\trecall={recall}\tspe={specificity}\tpre={precision}\n")
            i = i + 1
            time_elapsed1 = time.time() - since1
            print(time_elapsed1 // 60, time_elapsed1 % 60)
            
    df.to_csv('DTF_negResults.csv',index=False)  # index=False 表示不写入行索引
    time_elapsed = time.time() - since
    print(time_elapsed // 60, time_elapsed % 60)

DTF 80
1
(351, 351)
(325, 325)
0
1
2
3
4
5
6
7
8
9
(351, 351, 325)
14679.0
DTF
(351, 57) (351, 57) (325, 57)
(21137, 3)
(2349, 3)
torch.Size([23486, 3])
torch.Size([5872, 3])
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
tensor([0., 1., 1.,  ..., 0., 0., 1.]) tensor([1., 0., 1.,  ..., 0., 1., 0.])
DTF_new(
  (layers): ModuleList(
    (0): Linear(in_features=171, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
)


 19%|██████████████████████▌                                                                                              | 193/1000 [03:13<13:28,  1.00s/it]


Fold 1/5, Test Loss: 0.175125
tensor([[0.9987],
        [0.7194],
        [0.9925],
        ...,
        [0.0871],
        [0.0024],
        [0.0178]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DTF_0_times_0_foldscores.pkl
DTF
(351, 57) (351, 57) (325, 57)
(21137, 3)
(2349, 3)
torch.Size([23486, 3])
torch.Size([5872, 3])
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
tensor([0., 1., 1.,  ..., 0., 0., 1.]) tensor([1., 0., 1.,  ..., 0., 1., 0.])
DTF_new(
  (layers): ModuleList(
    (0): Linear(in_features=171, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  

 14%|████████████████▋                                                                                                    | 143/1000 [02:23<14:20,  1.00s/it]


Fold 2/5, Test Loss: 0.177468
tensor([[0.8733],
        [0.8491],
        [0.9285],
        ...,
        [0.7201],
        [0.0310],
        [0.0085]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DTF_0_times_1_foldscores.pkl
DTF
(351, 57) (351, 57) (325, 57)
(21137, 3)
(2349, 3)
torch.Size([23486, 3])
torch.Size([5872, 3])
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
tensor([0., 1., 1.,  ..., 0., 0., 1.]) tensor([1., 0., 1.,  ..., 0., 1., 0.])
DTF_new(
  (layers): ModuleList(
    (0): Linear(in_features=171, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  

...

Fold 3/5, Test Loss: 0.160517
tensor([[0.9555],
        [0.9610],
        [0.4573],
        ...,
        [0.0029],
        [0.0400],
        [0.0011]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DTF_4_times_2_foldscores.pkl
DTF
(351, 57) (351, 57) (325, 57)
(21137, 3)
(2349, 3)
torch.Size([23486, 3])
torch.Size([5872, 3])
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
(21137, 171) (2349, 171) (23486, 171) (5872, 171)
tensor([0., 1., 1.,  ..., 0., 0., 1.]) tensor([1., 0., 1.,  ..., 0., 1., 0.])
DTF_new(
  (layers): ModuleList(
    (0): Linear(in_features=171, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  

 17%|███████████████████▉                                                                                                 | 170/1000 [02:44<13:23,  1.03it/s]


Fold 4/5, Test Loss: 0.188903
tensor([[0.5910],
        [0.9983],
        [0.9938],
        ...,
        [0.1125],
        [0.0660],
        [0.0023]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DTF_4_times_3_foldscores.pkl
DTF
(351, 57) (351, 57) (325, 57)
(21139, 3)
(2349, 3)
torch.Size([23488, 3])
torch.Size([5870, 3])
(21139, 171) (2349, 171) (23488, 171) (5870, 171)
(21139, 171) (2349, 171) (23488, 171) (5870, 171)
tensor([1., 0., 1.,  ..., 0., 0., 1.]) tensor([1., 1., 1.,  ..., 1., 0., 0.])
DTF_new(
  (layers): ModuleList(
    (0): Linear(in_features=171, out_features=2048, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=2048, out_features=1024, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=512, out_features=1, bias=True)
    (10): Sigmoid()
  

 16%|██████████████████▊                                                                                                  | 161/1000 [02:24<12:35,  1.11it/s]

Fold 5/5, Test Loss: 0.172366
tensor([[9.0032e-01],
        [9.9936e-01],
        [9.9596e-01],
        ...,
        [2.3687e-04],
        [1.2943e-03],
        [2.9165e-02]], device='cuda:0') tensor([1., 1., 1.,  ..., 0., 0., 0.])
pred_score_pkl/DTF_4_times_4_foldscores.pkl
Times:	 5 :	 [[0.9852 0.9852 0.9469 0.9467 0.9501 0.9434 0.9438]]
final:	 [[0.9832 0.9836 0.9447 0.9445 0.9489 0.9401 0.9407]]
i=0	times=5	methods=DTF	msi=80	neg=1
auc=0.9836	aupr=0.9832	f1=0.9447	acc=0.9445	recall=0.9489	spe=0.9401	pre=0.9407

60.0 1.4734878540039062
60.0 1.6929316520690918
